In [2]:
#그룹1
################r 검증완료 #################
# =============================================================================
# 🚀 그룹 1: 통일된 함수명 - 마이리얼트립 크롤링 시스템 (리팩토링 완료)
# - 도시 정보를 UNIFIED_CITY_INFO로 통합하여 단일 소스로 관리
# =============================================================================

import pandas as pd
import warnings, os, time, shutil, urllib, random
warnings.filterwarnings(action='ignore')

import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용
from datetime import datetime    # 타임스탬프용

from PIL import Image
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "MIN_DELAY": 5,                  # 3 → 5초로 증가
    "MAX_DELAY": 12,                 # 8 → 12초로 증가
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    "SAVE_INTERMEDIATE": True,
    "MAX_PRODUCT_NAME_LENGTH": 30,
    "LONGER_DELAYS": True,           # 새로 추가
    "MEMORY_CLEANUP_INTERVAL": 5,    # 새로 추가
    "MAX_PRODUCTS_PER_CITY": 1,     # 2 → 10개로 증가⭐⭐⭐⭐⭐⭐⭐⭐⭐
    # 🆕 Gemini 지적사항 해결: USER_AGENT 추가
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36"
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["푸켓"]

# =============================================================================
# 📍 [최종 수정본] 단일 정보 소스 및 리팩토링된 함수
# =============================================================================

# 모든 도시의 상세 정보가 포함된 딕셔너리 (최종 확장본)
UNIFIED_CITY_INFO = {
    # 동남아시아
    "방콕": {"대륙": "아시아", "국가": "태국", "코드": "BKK"},
    "아유타야": {"대륙": "아시아", "국가": "태국", "코드": "BKK"}, # 방콕 공항 사용
    "치앙마이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"},
    "빠이": {"대륙": "아시아", "국가": "태국", "코드": "CNX"}, # 치앙마이 공항 사용
    "치앙라이": {"대륙": "아시아", "국가": "태국", "코드": "CEI"},
    "푸켓": {"대륙": "아시아", "국가": "태국", "코드": "HKT"},
    "피피": {"대륙": "아시아", "국가": "태국", "코드": "KBV"}, # 크라비 공항 사용
    "크라비": {"대륙": "아시아", "국가": "태국", "코드": "KBV"},
    "후아힌": {"대륙": "아시아", "국가": "태국", "코드": "HHQ"},
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르", "코드": "SIN"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩", "코드": "HKG"},
    "쿠알라룸푸르": {"대륙": "아시아", "국가": "말레이시아", "코드": "KUL"},
    "코타키나발루": {"대륙": "아시아", "국가": "말레이시아", "코드": "BKI"},
    "페낭": {"대륙": "아시아", "국가": "말레이시아", "코드": "PEN"},
    "랑카위": {"대륙": "아시아", "국가": "말레이시아", "코드": "LGK"},
    "세부": {"대륙": "아시아", "국가": "필리핀", "코드": "CEB"},
    "보홀": {"대륙": "아시아", "국가": "필리핀", "코드": "TAG"},
    "마닐라": {"대륙": "아시아", "국가": "필리핀", "코드": "MNL"},
    "보라카이": {"대륙": "아시아", "국가": "필리핀", "코드": "MPH"}, # 카티클란 공항
    "다낭": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"},
    "호이안": {"대륙": "아시아", "국가": "베트남", "코드": "DAD"}, # 다낭 공항 사용
    "후에": {"대륙": "아시아", "국가": "베트남", "코드": "HUI"},
    "호치민": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"},
    "무이네": {"대륙": "아시아", "국가": "베트남", "코드": "SGN"}, # 호치민 공항 사용
    "푸꾸옥": {"대륙": "아시아", "국가": "베트남", "코드": "PQC"},
    "나트랑": {"대륙": "아시아", "국가": "베트남", "코드": "CXR"},
    "하노이": {"대륙": "아시아", "국가": "베트남", "코드": "HAN"},
    "달랏": {"대륙": "아시아", "국가": "베트남", "코드": "DLI"},
    "발리": {"대륙": "아시아", "국가": "인도네시아", "코드": "DPS"},
    "프놈펜": {"대륙": "아시아", "국가": "캄보디아", "코드": "PNH"}, #아직 상품없음 25.0723
    "시엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"}, #아직 상품없음 25.0723
    "씨엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP"}, # 시엠립 동의어, #아직 상품없음 25.0723
    "비엔티안": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"},
    "방비엥": {"대륙": "아시아", "국가": "라오스", "코드": "VTE"}, # 비엔티안 공항 사용
    "루앙프라방": {"대륙": "아시아", "국가": "라오스", "코드": "LPQ"},

    # 동북아시아
    "도쿄": {"대륙": "아시아", "국가": "일본", "코드": "NRT"},
    "오사카": {"대륙": "아시아", "국가": "일본", "코드": "KIX"},
    "교토": {"대륙": "아시아", "국가": "일본", "코드": "KIX"}, # 오사카 공항 사용
    "나고야": {"대륙": "아시아", "국가": "일본", "코드": "NGO"},
    "후쿠오카": {"대륙": "아시아", "국가": "일본", "코드": "FUK"},
    "벳푸": {"대륙": "아시아", "국가": "일본", "코드": "OIT"}, # 오이타 공항 사용
    "오이타": {"대륙": "아시아", "국가": "일본", "코드": "OIT"},
    "구마모토": {"대륙": "아시아", "국가": "일본", "코드": "KMJ"},
    "오키나와": {"대륙": "아시아", "국가": "일본", "코드": "OKA"},
    "미야코지마": {"대륙": "아시아", "국가": "일본", "코드": "MMY"},
    "삿포로": {"대륙": "아시아", "국가": "일본", "코드": "CTS"},
    "타이베이": {"대륙": "아시아", "국가": "대만", "코드": "TPE"},
    "상하이": {"대륙": "아시아", "국가": "중국", "코드": "PVG"},
    "베이징": {"대륙": "아시아", "국가": "중국", "코드": "PEK"},
    "하이난(싼야)": {"대륙": "아시아", "국가": "중국", "코드": "SYX"},
    "마카오": {"대륙": "아시아", "국가": "마카오", "코드": "MFM"},
    
    # 남아시아
    "뉴델리": {"대륙": "아시아", "국가": "인도", "코드": "DEL"},
    "뭄바이": {"대륙": "아시아", "국가": "인도", "코드": "BOM"},
    "카트만두": {"대륙": "아시아", "국가": "네팔", "코드": "KTM"},
    "포카라": {"대륙": "아시아", "국가": "네팔", "코드": "PKR"},

    # 한국
    "서울": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "부산": {"대륙": "아시아", "국가": "대한민국", "코드": "PUS"},
    "제주": {"대륙": "아시아", "국가": "대한민국", "코드": "CJU"},
    "대구": {"대륙": "아시아", "국가": "대한민국", "코드": "TAE"},
    "광주": {"대륙": "아시아", "국가": "대한민국", "코드": "KWJ"},
    "여수": {"대륙": "아시아", "국가": "대한민국", "코드": "RSU"},
    "인천": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN"},
    "김포": {"대륙": "아시아", "국가": "대한민국", "코드": "GMP"},

    # 유럽
    "파리": {"대륙": "유럽", "국가": "프랑스", "코드": "CDG"},
    "런던": {"대륙": "유럽", "국가": "영국", "코드": "LHR"},
    "더블린": {"대륙": "유럽", "국가": "아일랜드", "코드": "DUB"},
    "로마": {"대륙": "유럽", "국가": "이탈리아", "코드": "FCO"},
    "피렌체": {"대륙": "유럽", "국가": "이탈리아", "코드": "FLR"},
    "베네치아": {"대륙": "유럽", "국가": "이탈리아", "코드": "VCE"},
    "밀라노": {"대륙": "유럽", "국가": "이탈리아", "코드": "MXP"},
    "바르셀로나": {"대륙": "유럽", "국가": "스페인", "코드": "BCN"},
    "마드리드": {"대륙": "유럽", "국가": "스페인", "코드": "MAD"},
    "세비야": {"대륙": "유럽", "국가": "스페인", "코드": "SVQ"},
    "그라나다": {"대륙": "유럽", "국가": "스페인", "코드": "GRX"},
    "이비자": {"대륙": "유럽", "국가": "스페인", "코드": "IBZ"},
    "리스본": {"대륙": "유럽", "국가": "포르투갈", "코드": "LIS"},
    "포르투": {"대륙": "유럽", "국가": "포르투갈", "코드": "OPO"},
    "프라하": {"대륙": "유럽", "국가": "체코", "코드": "PRG"},
    "비엔나": {"대륙": "유럽", "국가": "오스트리아", "코드": "VIE"},
    "취리히": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"},
    "인터라켄": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH"}, # 취리히 공항 사용
    "암스테르담": {"대륙": "유럽", "국가": "네덜란드", "코드": "AMS"},
    "브뤼셀": {"대륙": "유럽", "국가": "벨기에", "코드": "BRU"},
    "뮌헨": {"대륙": "유럽", "국가": "독일", "코드": "MUC"},
    "베를린": {"대륙": "유럽", "국가": "독일", "코드": "BER"},
    "프랑크푸르트": {"대륙": "유럽", "국가": "독일", "코드": "FRA"},
    "부다페스트": {"대륙": "유럽", "국가": "헝가리", "코드": "BUD"},
    "바르샤바": {"대륙": "유럽", "국가": "폴란드", "코드": "WAW"},
    "크라쿠프": {"대륙": "유럽", "국가": "폴란드", "코드": "KRK"},
    "아테네": {"대륙": "유럽", "국가": "그리스", "코드": "ATH"},
    "산토리니": {"대륙": "유럽", "국가": "그리스", "코드": "JTR"},
    "자그레브": {"대륙": "유럽", "국가": "크로아티아", "코드": "ZAG"},
    "두브로브니크": {"대륙": "유럽", "국가": "크로아티아", "코드": "DBV"},
    "코펜하겐": {"대륙": "유럽", "국가": "덴마크", "코드": "CPH"},
    "스톡홀름": {"대륙": "유럽", "국가": "스웨덴", "코드": "ARN"},
    "오슬로": {"대륙": "유럽", "국가": "노르웨이", "코드": "OSL"},
    "헬싱키": {"대륙": "유럽", "국가": "핀란드", "코드": "HEL"},
    "이스탄불": {"대륙": "유럽", "국가": "터키", "코드": "IST"},

    # 북미
    "뉴욕": {"대륙": "북미", "국가": "미국", "코드": "JFK"},
    "로스앤젤레스": {"대륙": "북미", "국가": "미국", "코드": "LAX"},
    "시카고": {"대륙": "북미", "국가": "미국", "코드": "ORD"},
    "하와이": {"대륙": "북미", "국가": "미국", "코드": "HNL"},
    "샌프란시스코": {"대륙": "북미", "국가": "미국", "코드": "SFO"},
    "라스베이거스": {"대륙": "북미", "국가": "미국", "코드": "LAS"},
    "워싱턴 D.C.": {"대륙": "북미", "국가": "미국", "코드": "IAD"},
    "보스턴": {"대륙": "북미", "국가": "미국", "코드": "BOS"},
    "시애틀": {"대륙": "북미", "국가": "미국", "코드": "SEA"},
    "밴쿠버": {"대륙": "북미", "국가": "캐나다", "코드": "YVR"},
    "토론토": {"대륙": "북미", "국가": "캐나다", "코드": "YYZ"},
    "칸쿤": {"대륙": "북미", "국가": "멕시코", "코드": "CUN"},

    # 오세아니아
    "시드니": {"대륙": "오세아니아", "국가": "호주", "코드": "SYD"},
    "멜버른": {"대륙": "오세아니아", "국가": "호주", "코드": "MEL"},
    "괌": {"대륙": "오세아니아", "국가": "괌", "코드": "GUM"},
    "사이판": {"대륙": "오세아니아", "국가": "북마리아나 제도", "코드": "SPN"},
    "오클랜드": {"대륙": "오세아니아", "국가": "뉴질랜드", "코드": "AKL"},
    
    # 중동
    "두바이": {"대륙": "아시아", "국가": "아랍에미리트", "코드": "DXB"},
}



print(f"✅ UNIFIED_CITY_INFO 로드 완료! {len(UNIFIED_CITY_INFO)}개 도시 지원")

# =============================================================================
# 🔧 핵심 함수들 - 단일 정보 소스(UNIFIED_CITY_INFO) 사용
# =============================================================================

def get_city_code(city_name):
    """도시명으로 공항 코드 반환 (UNIFIED_CITY_INFO 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        code = info.get("코드", city_name[:3].upper())
        return code
    return city_name[:3].upper()

def get_city_info(city_name):
    """통합된 도시 정보 가져오기 (사전 정의된 값만 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        return info["대륙"], info["국가"]
    else:
        # 정의되지 않은 도시에 대한 기본값
        return "기타", "기타"

def get_product_name(driver, url_type="Product"):
    """✅ 상품명 수집 (Offers 지원 강화)"""
    print(f"  📊 {url_type} 상품명 수집 중...")
    
    # Offers와 Products에 맞는 셀렉터들
    if url_type == "Offer":
        title_selectors = [
            (By.CSS_SELECTOR, "h1"),
            (By.CSS_SELECTOR, ".offer-title"),
            (By.CSS_SELECTOR, ".title"),
            (By.CSS_SELECTOR, "[class*='title']"),
            (By.XPATH, "//h1[contains(@class, 'title')]"),
            (By.XPATH, "//div[contains(@class, 'title')]//h1"),
            (By.XPATH, "//div[contains(@class, 'offer')]//h1"),
        ]
    else:
        title_selectors = [
            (By.CSS_SELECTOR, "h1"),
            (By.CSS_SELECTOR, ".product-title"),
            (By.XPATH, "//h1[contains(@class, 'title')]"),
            (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/h1")
        ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text.strip()
            if found_name:  # 빈 텍스트가 아닌 경우만
                return found_name
        except TimeoutException:
            continue
    
    # 최후의 수단: 페이지 타이틀 사용
    try:
        page_title = driver.title
        if page_title and '마이리얼트립' not in page_title:
            return page_title
    except:
        pass
    
    return f"상품명_수집실패_{url_type}"

def get_price(driver):
    """✅ 가격 수집 - 수정된 버전 (잘못된 텍스트 필터링 강화)"""
    print(f"  💰 가격 정보 수집 중...")
    
    # 마이리얼트립 실제 가격 셀렉터들
    price_selectors = [
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 87, 87)']"),
        (By.CSS_SELECTOR, "span[style*='color: rgb(255, 71, 71)']"),
        (By.CSS_SELECTOR, "span[style*='color: red']"),
        (By.CSS_SELECTOR, ".text-red"),
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        # 더 구체적인 XPath
        (By.XPATH, "//span[contains(text(), '원~') and contains(text(), ',') and string-length(text()) < 30]"),
        (By.XPATH, "//span[contains(text(), '원-') and contains(text(), ',') and string-length(text()) < 30]"),
        (By.XPATH, "//span[contains(text(), ',') and contains(text(), '원') and not(contains(text(), '인원')) and not(contains(text(), '최소'))]"),
    ]

    for selector_type, selector_value in price_selectors:
        try:
            price_elements = driver.find_elements(selector_type, selector_value)
            
            for price_element in price_elements:
                found_price = price_element.text.strip()
                
                if not found_price:
                    continue
                
                # 🔥 강력한 필터링: 가격이 아닌 텍스트들 제외
                invalid_keywords = [
                    '쿠폰', '받기', '다운', '할인', '적립', '포인트',
                    '최소', '인원', '명', '최대', '선택', '옵션',
                    '예약', '신청', '문의', '상담', '확인', '명부터',
                    '시간', '일정', '코스', '투어', '여행', '부터',
                    '언어', '가이드', '포함', '불포함', '이상',
                    '취소', '환불', '변경', '안내', '모집'
                ]
                
                if any(keyword in found_price for keyword in invalid_keywords):
                    continue
                
                # 길이 제한 (너무 긴 텍스트 제외)
                if len(found_price) > 30:
                    continue
                
                # 가격 패턴 확인
                import re
                price_patterns = [
                    r'\d{1,3}(?:,\d{3})+원[~-]?',  # 10,000원~ 형태
                    r'\d+,\d+원[~-]?',             # 간단한 천단위 구분
                    r'\d{4,}원[~-]?',              # 10000원~ 형태
                ]
                
                is_valid_price = any(re.search(pattern, found_price) for pattern in price_patterns)
                
                if is_valid_price and '원' in found_price:
                    print(f"    ✅ 유효한 가격 발견: '{found_price}'")
                    return found_price
                    
        except Exception:
            continue
    
    print(f"    ❌ 가격 정보를 찾을 수 없습니다")
    return "정보 없음"

def clean_price(price_text):
    """✅ 가격 정제 (기존: extract_clean_price → 새로운: clean_price) (공용 함수)"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    price_pattern = r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?'
    match = re.search(price_pattern, price_text)
    
    if match:
        return match.group(1) + "원"
    else:
        return price_text

def clean_rating(rating_text):
    """✅ 평점 정제 (기존: extract_clean_rating → 새로운: clean_rating) (공용 함수)"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return rating_text
    else:
        return rating_text

def download_image(driver, product_name, city_name, product_index):
    """✅ 이미지 다운로드 - 계층 구조 저장 (대륙/국가/도시/)"""
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'status': '이미지 저장 비활성화',
            'filename': '',
            'path': '',
            'relative_path': '',
            'size': 0
        }
        
    print(f"  🖼️ 대표 상품 이미지 다운로드 중...")
    
    # 더 광범위한 이미지 셀렉터들
    image_selectors = [
        (By.CSS_SELECTOR, ".main-image img"),
        (By.CSS_SELECTOR, ".hero-image img"),
        (By.CSS_SELECTOR, ".product-image img"),
        (By.CSS_SELECTOR, ".product-gallery img:first-child"),
        (By.CSS_SELECTOR, ".gallery img:first-child"),
        (By.CSS_SELECTOR, ".image-gallery img:first-child"),
        (By.CSS_SELECTOR, ".slider img:first-child"),
        (By.XPATH, "//img[@width and @height and (@width > '200' or @height > '200')]"),
        (By.XPATH, "//img[contains(@src, 'large') or contains(@src, 'big') or contains(@src, 'main')]"),
        (By.CSS_SELECTOR, "img[src*='cdn']"),
        (By.CSS_SELECTOR, "img[src*='cloudfront']"),
        (By.CSS_SELECTOR, "img[src*='amazonaws']"),
        (By.XPATH, "(//img[contains(@src, 'http') and not(contains(@src, 'icon')) and not(contains(@src, 'logo'))])[1]"),
        (By.XPATH, "//img[contains(@alt, '상품') or contains(@alt, '투어') or contains(@alt, '여행')]"),
    ]

    img_url = None
    for selector_type, selector_value in image_selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            for img_element in img_elements:
                try:
                    img_url = img_element.get_attribute('src')
                    if not img_url or not img_url.startswith('http'):
                        continue
                    exclude_patterns = ['logo', 'icon', 'banner', 'ad', 'avatar', 'profile', 'button', 'arrow', 'star', 'thumb', 'small', 'mini']
                    if any(pattern in img_url.lower() for pattern in exclude_patterns):
                        continue
                    try:
                        size = img_element.size
                        if size and (size.get('width', 0) < 100 or size.get('height', 0) < 100):
                            continue
                    except:
                        pass
                    print(f"    ✅ 이미지 URL 발견: {img_url[:50]}...")
                    break
                except Exception:
                    continue
            if img_url:
                break
        except Exception:
            continue

    if not img_url:
        print(f"    ❌ 이미지 URL을 찾을 수 없습니다")
        return {
            'status': '이미지 URL 없음',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'relative_path': '',
            'size': 0
        }

    # 🏗️ 계층 구조 폴더 생성 및 이미지 다운로드
    try:
        import urllib.request
        import os
        
        # 도시 정보 가져오기
        continent, country = get_city_info(city_name)
        city_code = get_city_code(city_name)
        img_filename = f"{city_code}_{product_index:03d}.jpg"
        
        # 📁 계층 구조 폴더 경로 생성: myrealtripthumb_img/대륙/국가/도시/
        base_folder = "myrealtripthumb_img"
        hierarchical_folder = os.path.join(base_folder, continent, country, city_name)
        
        # 폴더 생성 (계층 구조)
        os.makedirs(hierarchical_folder, exist_ok=True)
        print(f"    📁 계층 폴더 생성: {hierarchical_folder}")
        
        # 전체 파일 경로
        img_path = os.path.join(hierarchical_folder, img_filename)
        
        # 상대 경로 (CSV에 저장용)
        relative_path = os.path.join(continent, country, city_name, img_filename)
        
        # User-Agent 추가로 다운로드 성공률 높이기
        req = urllib.request.Request(
            img_url,
            headers={
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
            }
        )
        
        with urllib.request.urlopen(req, timeout=10) as response:
            with open(img_path, 'wb') as f:
                f.write(response.read())
        
        file_size = os.path.getsize(img_path)
        
        if file_size < 1024:
            os.remove(img_path)
            print(f"    ❌ 이미지 파일이 너무 작습니다 ({file_size} bytes)")
            return {
                'status': '파일 너무 작음',
                'filename': img_filename,
                'path': '',
                'relative_path': '',
                'size': 0
            }
        
        print(f"    ✅ 계층 구조 이미지 저장 완료! ({file_size:,} bytes)")
        print(f"    📍 저장 위치: {relative_path}")
        
        return {
            'status': '다운로드 완료',
            'filename': img_filename,
            'path': img_path,
            'relative_path': relative_path,
            'size': file_size
        }
        
    except Exception as e:
        print(f"    ⚠️ 이미지 다운로드 실패: {type(e).__name__}: {e}")
        return {
            'status': f'다운로드 실패: {type(e).__name__}',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'relative_path': '',
            'size': 0
        }
    else:
        return {
            'status': '이미지 없음',
            'filename': f"{get_city_code(city_name)}_{product_index:03d}.jpg",
            'path': '',
            'size': 0
        }

def save_results(products_data):
    """✅ 데이터 저장 - UTF-8 인코딩 강화 버전 (기존: save_myrealtrip_data → 새로운: save_results)"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    if not products_data:
        print("❌ 저장할 데이터가 없습니다")
        return None
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    city_name = products_data[0]['도시'] if products_data else 'unknown'
    
    df = pd.DataFrame(products_data)
    
    try:
        # 🔥 핵심 수정 1: 계층적 구조로 저장
        if city_name in UNIFIED_CITY_INFO:
            continent = UNIFIED_CITY_INFO[city_name]["대륙"]
            country = UNIFIED_CITY_INFO[city_name]["국가"]
            
            # 계층적 경로 생성
            hierarchical_path = f"data/{continent}/{country}/myrealtrip_{city_name}_complete.csv"
            
            # 🔥 핵심 수정 2: 폴더 강제 생성
            os.makedirs(os.path.dirname(hierarchical_path), exist_ok=True)
            
            # 🔥 핵심 수정 3: UTF-8 강제 인코딩 (BOM 포함)
            df.to_csv(hierarchical_path, index=False, encoding='utf-8-sig')
            print(f"📁 계층적 CSV 저장 완료: {hierarchical_path}")
            csv_path = hierarchical_path
        else:
            # 폴백: 기존 방식 (알 수 없는 도시)
            csv_path = f"myrealtrip_{city_name}_products_{len(products_data)}개_{timestamp}.csv"
            df.to_csv(csv_path, index=False, encoding='utf-8-sig')
            print(f"📁 기본 CSV 저장 완료: {csv_path}")
        
        # 메타데이터 저장
        metadata = {
            "myrealtrip": {
                "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "product_count": len(products_data),
                "status": "success",
                "csv_path": csv_path,
                "city": city_name,
                "encoding": "utf-8-sig",  # 🆕 인코딩 정보 추가
                "data_integrity": "verified"  # 🆕 무결성 확인 정보
            }
        }
        
        try:
            # 🔥 핵심 수정 4: config 폴더 강제 생성
            os.makedirs("config", exist_ok=True)
            with open('config/data_metadata.json', 'w', encoding='utf-8') as f:
                json.dump(metadata, f, ensure_ascii=False, indent=2)
            print(f"📁 메타데이터 저장 완료: config/data_metadata.json")
        except Exception as e:
            print(f"⚠️ 메타데이터 저장 실패: {e}")
        
        return csv_path
        
    except Exception as e:
        print(f"❌ 메인 저장 실패: {e}")
        # 🔥 핵심 수정 5: 긴급 백업 시스템
        try:
            emergency_path = f"emergency_save_{city_name}_{timestamp}.csv"
            df.to_csv(emergency_path, index=False, encoding='utf-8-sig')
            print(f"🚨 긴급 백업 저장 성공: {emergency_path}")
            print(f"💡 나중에 {emergency_path}를 올바른 위치로 이동하세요!")
            return emergency_path
        except Exception as emergency_error:
            print(f"🚨 긴급 백업도 실패: {emergency_error}")
            return None

# =============================================================================
# 🚀 Phase 2: 확장성 개선 시스템 (리팩토링된 버전)
# =============================================================================

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장 - 수정된 버전 (UTF-8 강화)"""
    enhanced_city_data = {
        "version": "3.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": {},
        "total_cities": len(UNIFIED_CITY_INFO)
    }

    for city_name, info in UNIFIED_CITY_INFO.items():
        enhanced_city_data["cities"][city_name] = {
            "code": info.get("코드", "N/A"),
            "continent": info.get("대륙", "기타"),
            "country": info.get("국가", "기타")
        }
    
    try:
        # 🔥 핵심 수정: config 폴더 강제 생성 + UTF-8 저장
        os.makedirs("config", exist_ok=True)
        with open('config/city_codes.json', 'w', encoding='utf-8') as f:
            json.dump(enhanced_city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ city_codes.json 파일 생성 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드 (UNIFIED_CITY_INFO와 동기화)"""
    if not os.path.exists('config/city_codes.json'):
        print("📝 city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
    
    try:
        with open('config/city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        loaded_cities = city_data.get("cities", {})
        
        for city, info in loaded_cities.items():
            if city not in UNIFIED_CITY_INFO:
                 UNIFIED_CITY_INFO[city] = {
                     "대륙": info.get("continent"),
                     "국가": info.get("country"),
                     "코드": info.get("code")
                 }
        
        print(f"✅ city_codes.json 로드 및 동기화 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 코드의 UNIFIED_CITY_INFO를 사용합니다.")

def add_new_city(city_name, airport_code, continent="기타", country="기타", update_file=True):
    """새로운 도시를 UNIFIED_CITY_INFO에 추가하는 함수"""
    global UNIFIED_CITY_INFO
    
    if city_name not in UNIFIED_CITY_INFO:
        UNIFIED_CITY_INFO[city_name] = {
            "대륙": continent,
            "국가": country,
            "코드": airport_code
        }
        print(f"✅ 메모리에 추가: {city_name} → {airport_code} ({continent}, {country})")
        if update_file:
            create_city_codes_file()
        return True
    else:
        print(f"이미 존재하는 도시입니다: {city_name}")
        return False

def show_supported_cities():
    """지원하는 도시 목록 표시 (UNIFIED_CITY_INFO 기반)"""
    print("\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    cities_by_continent = {}
    for city, info in UNIFIED_CITY_INFO.items():
        continent = info.get("대륙", "기타")
        if continent not in cities_by_continent:
            cities_by_continent[continent] = []
        cities_by_continent[continent].append(city)

    for continent, cities in sorted(cities_by_continent.items()):
        print(f"\n📍 {continent}:")
        for city in sorted(cities):
            code = UNIFIED_CITY_INFO[city].get("코드", "N/A")
            print(f"   {city} → {code}")
    
    print(f"\n📊 총 {len(UNIFIED_CITY_INFO)}개 도시 지원")
    print("="*50)

def validate_city(city_name):
    """도시명 유효성 검사 (UNIFIED_CITY_INFO 기반)"""
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in UNIFIED_CITY_INFO:
        code = UNIFIED_CITY_INFO[city_name].get("코드", "N/A")
        return True, f"지원하는 도시입니다. ({code})"
    
    similar_cities = [c for c in UNIFIED_CITY_INFO if city_name.lower() in c.lower() or c.lower() in city_name.lower()]
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. 새로 추가하시려면 add_new_city() 함수를 사용하세요."

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    global CONFIG
    
    scalability_config = {
        "AUTO_LOAD_CITIES": True,
        "AUTO_SAVE_NEW_CITIES": True,
        "ENABLE_MULTI_CITY": False,
        "CITY_PROCESSING_ORDER": "sequential",
        "BACKUP_OLD_DATA": True,
        "DATA_RETENTION_DAYS": 30,
        "ENABLE_CITY_VALIDATION": True,
        "ENABLE_DUPLICATE_CHECK": True,
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

def initialize_file_system():
    """파일 시스템 초기화 및 설정 (리팩토링된 버전)"""
    print("🔧 Phase 2: 확장성 개선 시스템 초기화...")
    
    update_config_for_scalability()
    
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        load_city_codes_from_file()
    
    print("✅ Phase 2 시스템 초기화 완료!")
    return True

# =============================================================================
# 🛠️ 기존 유틸리티 함수들 (Phase 2 시스템과 함께 유지)
# =============================================================================

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG["SAVE_INTERMEDIATE"]:
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

def make_user_agent(ua, is_mobile):
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        shutil.rmtree(raw_path)
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
    except Exception as e:
        print('\n',"-"*50,"\n","-"*50,"\n")
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        print('https://appfollow.tistory.com/102')
        print('\n',"-"*50,"\n","-"*50,"\n")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """메인 페이지로 이동"""
    driver.get("https://www.myrealtrip.com/experiences/")
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 입력"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "input[placeholder*='어디로']"),
        (By.CSS_SELECTOR, "input[type='text']"),
        (By.XPATH, "//input[contains(@placeholder, '어디로')]"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/input")
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    search_input.clear()
    search_input.send_keys(city_name)
    time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True

def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "button[type='submit']"),
        (By.CSS_SELECTOR, ".search-btn"),
        (By.XPATH, "//button[contains(@class, 'search')]"),
        (By.XPATH, "//img[contains(@alt, '검색')]//parent::*"),
        (By.XPATH, "/html/body/main/div/div[2]/section[1]/div[1]/div/div/div/img")
    ]

    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭 (안정성 강화)"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            driver.execute_script("arguments[0].click();", view_all_button)
            
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MIN_DELAY"]+3))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
        
    return True

def collect_all_24_urls(driver):
    """✅ 모든 상품 URL 수집 - 중복 방지 강화 + 안정성 개선"""
    print("  📊 향상된 URL 수집 시스템 시작...")
    
    all_urls = []
    duplicate_count = 0
    
    try:
        # 🔥 핵심 수정 1: 페이지 로딩 안정화
        time.sleep(random.uniform(3, 6))
        
        # 🔥 핵심 수정 2: 더 포괄적인 셀렉터들
        url_selectors = [
            # 기본 experiences 링크들
            "a[href*='/experiences/']",
            "a[href*='/experience/']", 
            # 상품 컨테이너 내 링크들
            ".product-item a[href]",
            ".experience-card a[href]",
            ".product-card a[href]",
            # 리스트 항목들
            "li a[href*='/experiences/']",
            "div[class*='product'] a[href]",
            "div[class*='experience'] a[href]",
            # 일반적인 링크들
            "a[href][title*='투어']",
            "a[href][title*='여행']",
        ]
        
        print(f"    🔍 {len(url_selectors)}개 셀렉터로 URL 수집 중...")
        
        # 🔥 핵심 수정 3: 셀렉터별로 순차 수집
        for selector_idx, selector in enumerate(url_selectors, 1):
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                print(f"      - 셀렉터 {selector_idx}: {len(elements)}개 요소 발견")
                
                for element in elements:
                    try:
                        url = element.get_attribute('href')
                        if url and _is_valid_url(url):
                            # 🔥 핵심 수정 4: 중복 제거 강화
                            normalized_url = _normalize_url(url)
                            if not any(_normalize_url(existing) == normalized_url for existing in all_urls):
                                all_urls.append(url)
                            else:
                                duplicate_count += 1
                                
                    except Exception:
                        continue
                        
                # 충분한 URL이 수집되면 조기 종료
                if len(all_urls) >= 30:
                    print(f"      ✅ 충분한 URL 수집 완료 ({len(all_urls)}개)")
                    break
                    
            except Exception as e:
                print(f"      ⚠️ 셀렉터 {selector_idx} 실패: {type(e).__name__}")
                continue
        
        # 🔥 핵심 수정 5: Offers도 별도로 수집
        offers_count = _collect_offers(driver, all_urls)
        
        # 🔥 핵심 수정 6: 최종 검증 및 정리
        validated_urls = []
        invalid_count = 0
        
        for url in all_urls:
            if _final_validation(url):
                validated_urls.append(url)
            else:
                invalid_count += 1
        
        # 결과 출력
        print(f"  📊 URL 수집 완료!")
        print(f"    🔗 Products: {len(validated_urls) - offers_count}개")
        print(f"    🎯 Offers: {offers_count}개") 
        print(f"    🚫 중복 제거: {duplicate_count}개")
        print(f"    ❌ 유효하지 않음: {invalid_count}개")
        print(f"    ✅ 최종 유효 URL: {len(validated_urls)}개")
        
        return validated_urls
        
    except Exception as e:
        print(f"  ❌ URL 수집 중 오류: {type(e).__name__}: {e}")
        return all_urls if all_urls else []

def _is_valid_url(url):
    """URL 유효성 1차 검사"""
    if not url or not url.startswith('http'):
        return False
    
    # 기본 유효성 검사
    valid_patterns = [
        '/experiences/',
        '/offers/',
        'myrealtrip.com'
    ]
    
    return any(pattern in url for pattern in valid_patterns)

def _normalize_url(url):
    """URL 정규화 (중복 제거용)"""
    try:
        # 쿼리 파라미터 제거
        import urllib.parse
        parsed = urllib.parse.urlparse(url)
        normalized = f"{parsed.scheme}://{parsed.netloc}{parsed.path}"
        
        # 끝의 슬래시 제거
        return normalized.rstrip('/')
        
    except:
        return url

def _collect_offers(driver, existing_urls):
    """Offers 전용 수집"""
    offers_count = 0
    
    try:
        offer_selectors = [
            "a[href*='/offers/']",
            "div[class*='offer'] a[href]",
            ".offer-card a[href]"
        ]
        
        for selector in offer_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                for element in elements:
                    try:
                        url = element.get_attribute('href')
                        if url and '/offers/' in url:
                            normalized = _normalize_url(url)
                            if not any(_normalize_url(existing) == normalized for existing in existing_urls):
                                existing_urls.append(url)
                                offers_count += 1
                    except:
                        continue
            except:
                continue
                
    except Exception as e:
        print(f"    ⚠️ Offers 수집 실패: {type(e).__name__}")
    
    return offers_count

def _final_validation(url):
    """최종 URL 검증"""
    try:
        # 길이 체크
        if len(url) > 500:
            return False
            
        # 필수 패턴 체크
        required_patterns = ['myrealtrip', '/experiences/', '/offers/']
        if not any(pattern in url for pattern in required_patterns):
            return False
            
        # 제외 패턴 체크  
        exclude_patterns = ['javascript:', 'mailto:', '#', 'tel:']
        if any(pattern in url.lower() for pattern in exclude_patterns):
            return False
            
        return True
        
    except:
        return False

def get_rating(driver):
    """평점 정보 수집 (기존 함수 유지)"""
    rating_selectors = [
        (By.CSS_SELECTOR, ".rating"),
        (By.CSS_SELECTOR, "[class*='rating']"),
        (By.XPATH, "//span[contains(@class, 'rating')]"),
        (By.XPATH, "/html/body/div[1]/main/div[1]/section/div[1]/span/span[2]")
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            rating_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_rating = rating_element.text
            time.sleep(random.uniform(2, 4))
            return found_rating
        except TimeoutException:
            continue
    
    return "정보 없음"

def get_review_count(driver):
    """리뷰 수 정보 수집"""
    print(f"  📝 리뷰 수 정보 찾는 중...")
    review_count_selectors = [
        (By.XPATH, "//span[contains(text(), '리뷰')]"),
        (By.XPATH, "//span[contains(text(), 'review')]"),
        (By.XPATH, "//span[contains(text(), '후기')]"),
        (By.XPATH, "//span[contains(text(), '개')]"),
        (By.XPATH, "//span[contains(text(), '건')]"),
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            review_element = WebDriverWait(driver, 3).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            review_text = review_element.text.strip()
            
            review_keywords = ['리뷰', '후기', 'review', '개', '건']
            has_number = any(char.isdigit() for char in review_text)
            has_keyword = any(keyword in review_text.lower() for keyword in review_keywords)
            
            if has_number and has_keyword and len(review_text) < 50:
                print(f"  ✅ 리뷰 수 정보 발견: {review_text}")
                return review_text
                
        except TimeoutException:
            continue

    print(f"  ℹ️ 리뷰 수 정보를 찾을 수 없습니다.")
    return ""

def get_language(driver):
    """언어 정보 수집 - 수정된 버전 (후기 내용 제외)"""
    print(f"  🌐 언어 정보 찾는 중...")
    
    # 언어 정보가 있을 가능성이 높은 셀렉터들
    language_selectors = [
        # 언어 섹션 직접 타겟팅
        (By.XPATH, "//dt[contains(text(), '언어')]/following-sibling::dd"),
        (By.XPATH, "//span[contains(@class, 'language')]"),
        (By.XPATH, "//div[contains(@class, 'language')]//span"),
        # 짧은 언어 텍스트만 (후기 제외)
        (By.XPATH, "//span[contains(text(), '한국어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '영어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '중국어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), '일본어') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), 'Korean') and string-length(text()) < 20]"),
        (By.XPATH, "//span[contains(text(), 'English') and string-length(text()) < 20]"),
        # 언어 아이콘 근처의 텍스트
        (By.XPATH, "//i[contains(@class, 'language')]/following-sibling::span"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            language_elements = driver.find_elements(selector_type, selector_value)
            
            for language_element in language_elements:
                language_text = language_element.text.strip()
                
                if not language_text:
                    continue
                
                # 🔥 강력한 필터링: 후기나 긴 텍스트 제외
                # 길이 제한 (언어 정보는 보통 짧음)
                if len(language_text) > 50:
                    continue
                
                # 후기 관련 키워드 제외
                review_keywords = [
                    '후기', '리뷰', '평가', '별점', '추천', '만족',
                    '여행', '투어', '경험', '좋았', '나빴', '최고',
                    '다음에', '또', '재방문', '친절', '서비스',
                    '가격', '시간', '일정', '코스', '가이드',
                    '예약', '신청', '문의', '확인', '취소'
                ]
                
                if any(keyword in language_text for keyword in review_keywords):
                    continue
                
                # 언어 키워드 확인
                language_keywords = [
                    '언어', '한국어', '영어', '중국어', '일본어', 
                    'Korean', 'English', 'Chinese', 'Japanese',
                    '가능', '지원', '제공'
                ]
                
                if any(keyword in language_text for keyword in language_keywords):
                    print(f"    ✅ 언어 정보 발견: {language_text}")
                    return language_text
                    
        except Exception:
            continue

    print(f"    ℹ️ 언어 정보를 찾을 수 없습니다.")
    return ""

# =============================================================================
# Phase 2 시스템 자동 실행
# =============================================================================

try:
    initialize_file_system()
    
    # quick_add_cities()는 모든 도시가 UNIFIED_CITY_INFO에 포함되어 더 이상 필요 없음
    
    show_supported_cities()
    
    print("\n🎉 Phase 2: 확장성 개선 완료!")
    print("💡 이제 이런 기능들을 사용할 수 있습니다:")
    print("   - add_new_city('제주도', 'CJU', '아시아', '대한민국')  # 새 도시 추가")
    print("   - show_supported_cities()        # 지원 도시 목록")
    print("   - validate_city('방콕')          # 도시 유효성 검사")
    
except Exception as e:
    print(f"❌ Phase 2 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")

print("\n" + "="*60)
print("✅ 그룹 1 완료: 모든 함수가 정의되었습니다!")
print("="*60)
print("🔧 핵심 함수명 변경 완료:")
print("   get_product_name_by_type() → get_product_name()")
print("   get_price_fixed() → get_price()")
print("   download_image_improved_fixed() → download_image()")
print("   extract_clean_price() → clean_price()")
print("   extract_clean_rating() → clean_rating()")
print("   save_myrealtrip_data() → save_results()")
print("="*60)
print(f"🔢 현재 설정: {CONFIG['MAX_PRODUCTS_PER_CITY']}개 상품 크롤링")
print(f"🏙️ 검색 도시: {CITIES_TO_SEARCH}")
print("🎯 다음: 그룹 2,3,4에서 함수 호출을 통일된 함수명으로 변경하세요!")
print("🚨 Gemini 지적사항 모두 해결 완료! 안전하게 실행 가능합니다!")


🔧 Selenium 버전: 4.25.0
✅ UNIFIED_CITY_INFO 로드 완료! 116개 도시 지원
🔧 Phase 2: 확장성 개선 시스템 초기화...
⚙️ CONFIG 확장성 설정 업데이트 완료!
✅ city_codes.json 로드 및 동기화 완료! (116개 도시)
📅 마지막 업데이트: 2025-07-17 23:36:15
✅ Phase 2 시스템 초기화 완료!

🌍 지원하는 도시 목록:

📍 북미:
   뉴욕 → JFK
   라스베이거스 → LAS
   로스앤젤레스 → LAX
   밴쿠버 → YVR
   보스턴 → BOS
   샌프란시스코 → SFO
   시애틀 → SEA
   시카고 → ORD
   워싱턴 D.C. → IAD
   칸쿤 → CUN
   토론토 → YYZ
   하와이 → HNL

📍 아시아:
   광주 → KWJ
   교토 → KIX
   구마모토 → KMJ
   김포 → GMP
   나고야 → NGO
   나트랑 → CXR
   뉴델리 → DEL
   다낭 → DAD
   달랏 → DLI
   대구 → TAE
   도쿄 → NRT
   두바이 → DXB
   랑카위 → LGK
   루앙프라방 → LPQ
   마닐라 → MNL
   마카오 → MFM
   무이네 → SGN
   뭄바이 → BOM
   미야코지마 → MMY
   발리 → DPS
   방비엥 → VTE
   방콕 → BKK
   베이징 → PEK
   벳푸 → OIT
   보라카이 → MPH
   보홀 → TAG
   부산 → PUS
   비엔티안 → VTE
   빠이 → CNX
   삿포로 → CTS
   상하이 → PVG
   서울 → ICN
   세부 → CEB
   시엠립 → REP
   싱가포르 → SIN
   씨엠립 → REP
   아유타야 → BKK
   여수 → RSU
   오사카 → KIX
   오이타 → OIT
   오키나와 → OKA
   인천 → ICN
   제주 → CJU
   치앙라이 → CEI
   치앙마이 → CNX
   카트만두 → KTM

In [3]:

##1-1 새로운 그룹코드 테스트중
# =============================================================================
# 🛡️ 수정된 안전한 계층적 폴더 구조 생성 시스템
# - UTF-8 인코딩 강화
# - 오류 처리 개선
# - 기존 파일 보호 강화
# =============================================================================

import os
import glob
from datetime import datetime

def create_safe_hierarchical_folders():
    """안전한 계층적 폴더 구조 생성 - 수정된 버전"""
    print("🛡️ 안전한 계층적 폴더 구조 생성 시작...")
    print("📋 원칙: 기존 파일들은 절대 건드리지 않음")
    
    created_folders = []
    errors = []
    
    try:
        # 1. 기본 구조 생성
        print("\n📁 Step 1: 기본 구조 생성...")
        basic_folders = [
            "config",           # 설정 파일 보관
            "data",            # 새로운 계층적 데이터 저장소
            "data/history",    # 히스토리 보관 (sessions → history로 변경됨)
            "temp",            # 임시 파일들
            "temp/cookies"     # 쿠키 임시 저장
        ]
        
        for folder in basic_folders:
            try:
                os.makedirs(folder, exist_ok=True)
                created_folders.append(folder)
                print(f"  ✅ {folder}/ 생성 완료")
            except Exception as e:
                error_msg = f"폴더 생성 실패 {folder}: {e}"
                errors.append(error_msg)
                print(f"  ❌ {error_msg}")
        
        # 2. 대륙/국가 계층 구조 생성
        print(f"\n🌍 Step 2: 대륙/국가 계층 구조 생성...")
        print(f"📊 총 {len(UNIFIED_CITY_INFO)}개 도시에서 대륙/국가 정보 추출...")
        
        # 대륙/국가 조합 수집
        continent_country_pairs = set()
        for city_name, info in UNIFIED_CITY_INFO.items():
            continent = info.get("대륙", "기타")
            country = info.get("국가", "기타")
            continent_country_pairs.add((continent, country))
        
        print(f"🗺️ 발견된 대륙/국가 조합: {len(continent_country_pairs)}개")
        
        # 계층 구조 생성
        continents_created = set()
        countries_created = set()
        
        for continent, country in sorted(continent_country_pairs):
            try:
                # 대륙 폴더 생성
                continent_path = f"data/{continent}"
                if continent not in continents_created:
                    os.makedirs(continent_path, exist_ok=True)
                    continents_created.add(continent)
                    print(f"  🌏 {continent}/ 생성 완료")
                
                # 국가 폴더 생성
                country_path = f"data/{continent}/{country}"
                country_key = f"{continent}/{country}"
                if country_key not in countries_created:
                    os.makedirs(country_path, exist_ok=True)
                    countries_created.add(country_key)
                    print(f"    🏳️ {continent}/{country}/ 생성 완료")
                    
            except Exception as e:
                error_msg = f"계층 구조 생성 실패 {continent}/{country}: {e}"
                errors.append(error_msg)
                print(f"    ❌ {error_msg}")
        
        # 3. 생성된 구조 검증
        print(f"\n🔍 Step 3: 생성된 구조 검증...")
        verification_results = verify_folder_structure()
        
        # 4. 기존 파일 무결성 확인
        print(f"\n🛡️ Step 4: 기존 파일 무결성 확인...")
        integrity_check = check_existing_files_integrity()
        
        # 5. 결과 요약
        print(f"\n📊 생성 결과:")
        print(f"  📁 생성된 기본 폴더: {len(basic_folders)}개")
        print(f"  🌍 생성된 대륙: {len(continents_created)}개")
        print(f"  🏳️ 생성된 국가: {len(countries_created)}개")
        print(f"  ❌ 오류: {len(errors)}개")
        
        if len(errors) == 0:
            print(f"\n🎉 계층적 폴더 구조 생성 완료!")
            print(f"📁 모든 폴더가 안전하게 생성됨")
        else:
            print(f"\n⚠️ 일부 오류 발생:")
            for error in errors[:5]:  # 최대 5개만 표시
                print(f"   - {error}")
        
        return {
            "success": len(errors) == 0,
            "created_folders": created_folders,
            "errors": errors,
            "continents": len(continents_created),
            "countries": len(countries_created)
        }
        
    except Exception as e:
        error_msg = f"전체 프로세스 실패: {e}"
        print(f"\n❌ {error_msg}")
        return {
            "success": False,
            "created_folders": created_folders,
            "errors": [error_msg],
            "continents": 0,
            "countries": 0
        }

def verify_folder_structure():
    """생성된 폴더 구조 검증"""
    print("   🔍 폴더 구조 검증 중...")
    
    # 필수 폴더 확인
    required_folders = ["config", "data", "data/history", "temp"]
    missing_folders = []
    
    for folder in required_folders:
        if not os.path.exists(folder):
            missing_folders.append(folder)
    
    if missing_folders:
        print(f"   ⚠️ 누락된 폴더: {', '.join(missing_folders)}")
        return False
    else:
        print(f"   ✅ 모든 필수 폴더 존재 확인")
        
    # 대륙 폴더 확인
    if os.path.exists("data"):
        continents = [f for f in os.listdir("data") 
                     if os.path.isdir(f"data/{f}") and f != "history"]
        print(f"   📊 생성된 대륙 폴더: {len(continents)}개")
        
        # 국가 폴더 확인
        total_countries = 0
        for continent in continents:
            continent_path = f"data/{continent}"
            if os.path.exists(continent_path):
                countries = [f for f in os.listdir(continent_path) 
                           if os.path.isdir(f"{continent_path}/{f}")]
                total_countries += len(countries)
        
        print(f"   📊 생성된 국가 폴더: {total_countries}개")
        
    return True

def check_existing_files_integrity():
    """기존 파일들의 무결성 확인"""
    print("   🛡️ 기존 파일 무결성 검사...")
    
    # 중요한 기존 파일들 확인
    important_files = [
        "test_4.ipynb",
        # CSV 파일들은 data/ 폴더로 이동되었을 수 있으므로 체크하지 않음
    ]
    
    preserved_files = []
    for file in important_files:
        if os.path.exists(file):
            preserved_files.append(file)
    
    print(f"   ✅ 보존된 중요 파일: {len(preserved_files)}개")
    
    # 이미지 폴더 확인
    if os.path.exists("myrealtripthumb_img"):
        print(f"   ✅ 이미지 폴더 그대로 유지")
    
    return True

def show_hierarchical_structure():
    """생성된 계층적 구조 시각화 - 수정된 버전"""
    print("\n🗂️ 생성된 계층적 폴더 구조:")
    print("=" * 60)
    
    try:
        print("📁 mikael_project/ (프로젝트 루트)")
        
        # config 폴더
        if os.path.exists("config"):
            print("├── 📁 config/ (설정 파일 보관)")
            config_files = [f for f in os.listdir("config") if f.endswith('.json')]
            for i, file in enumerate(config_files):
                prefix = "│   └──" if i == len(config_files)-1 else "│   ├──"
                print(f"{prefix} 📄 {file}")
        
        # data 폴더
        print("├── 📁 data/ (새로운 계층적 저장소)")
        
        # history 폴더
        if os.path.exists("data/history"):
            print("│   ├── 📁 history/ (크롤링 히스토리 보관)")
        
        # 대륙 폴더들
        if os.path.exists("data"):
            continents = [f for f in os.listdir("data") 
                         if os.path.isdir(f"data/{f}") and f != "history"]
            
            for i, continent in enumerate(sorted(continents)):
                is_last_continent = i == len(continents) - 1
                continent_prefix = "│   └──" if is_last_continent else "│   ├──"
                print(f"{continent_prefix} 📁 {continent}/")
                
                # 국가 폴더들 (일부만 표시)
                continent_path = f"data/{continent}"
                if os.path.exists(continent_path):
                    countries = [f for f in os.listdir(continent_path) 
                               if os.path.isdir(f"{continent_path}/{f}")][:3]  # 처음 3개만
                    
                    for j, country in enumerate(sorted(countries)):
                        is_last_country = j == len(countries) - 1
                        if is_last_continent:
                            country_prefix = "       └──" if is_last_country else "       ├──"
                        else:
                            country_prefix = "│      └──" if is_last_country else "│      ├──"
                        print(f"{country_prefix} 📁 {country}/")
                    
                    # 더 많은 국가가 있으면 표시
                    total_countries = len([f for f in os.listdir(continent_path) 
                                         if os.path.isdir(f"{continent_path}/{f}")])
                    if total_countries > 3:
                        if is_last_continent:
                            print(f"       └── ... (총 {total_countries}개국)")
                        else:
                            print(f"│      └── ... (총 {total_countries}개국)")
        
        # temp 폴더
        print("├── 📁 temp/ (임시 파일들)")
        if os.path.exists("temp/cookies"):
            print("│   └── 📁 cookies/")
        
        # 기존 폴더들
        if os.path.exists("myrealtripthumb_img"):
            print("└── 📁 myrealtripthumb_img/ (그대로 유지)")
        
        print("=" * 60)
        print("🎯 결과: 체계적인 계층 구조 완성!")
        
    except Exception as e:
        print(f"❌ 구조 표시 실패: {e}")

def test_safe_folder_creation():
    """안전한 폴더 생성 테스트 - 수정된 버전"""
    print("🧪 안전한 계층적 폴더 구조 생성 테스트")
    print("=" * 60)
    
    print("📋 실행 전 확인사항:")
    print("   ✅ 기존 그룹 1이 실행된 상태 (UNIFIED_CITY_INFO 필요)")
    print("   ✅ 기존 파일들이 있는 상태에서 실행")
    
    # 1. UNIFIED_CITY_INFO 확인
    if 'UNIFIED_CITY_INFO' not in globals():
        print("❌ UNIFIED_CITY_INFO가 없습니다!")
        print("💡 먼저 그룹 1을 실행해주세요!")
        return False
    
    print(f"✅ UNIFIED_CITY_INFO 확인: {len(UNIFIED_CITY_INFO)}개 도시")
    
    # 2. 폴더 생성 실행
    print("\n🚀 폴더 구조 생성 실행...")
    result = create_safe_hierarchical_folders()
    
    # 3. 구조 시각화
    if result["success"]:
        show_hierarchical_structure()
        print(f"\n🎯 다음 단계: config/crawling_state.json 시스템 구축")
        print(f"🎉 Step 1 완료! 다음 단계 준비됨")
    else:
        print(f"\n⚠️ 일부 오류 발생. 기존 파일들은 안전하게 보존됨.")
    
    return result["success"]

# =============================================================================
# 🚀 실행부
# =============================================================================

print("=" * 70)
print("🛡️ 수정된 안전한 계층적 폴더 구조 생성 시스템")
print("=" * 70)
print("📋 주요 개선사항:")
print("   🔤 UTF-8 인코딩 안정성 강화")
print("   🛡️ 기존 파일 보호 강화")
print("   ⚡ 오류 처리 개선")
print("   📊 상세한 검증 및 리포팅")
print("=" * 70)

print("\n🚀 자동 실행 시작!\n")

# 자동 실행
result = test_safe_folder_creation()

if result:
    print("\n✅ 수정된 1-1 코드 실행 완료!")
    print("🎯 이제 다음 단계들을 순서대로 실행하세요:")
    print("   1-2: 파일 통합 (선택사항)")  
    print("   1-3: 작전 일지 시스템")
    print("   1-4: 지능형 크롤러")
else:
    print("\n⚠️ 일부 문제 발생, 하지만 기존 파일들은 안전합니다.")

🛡️ 수정된 안전한 계층적 폴더 구조 생성 시스템
📋 주요 개선사항:
   🔤 UTF-8 인코딩 안정성 강화
   🛡️ 기존 파일 보호 강화
   ⚡ 오류 처리 개선
   📊 상세한 검증 및 리포팅

🚀 자동 실행 시작!

🧪 안전한 계층적 폴더 구조 생성 테스트
📋 실행 전 확인사항:
   ✅ 기존 그룹 1이 실행된 상태 (UNIFIED_CITY_INFO 필요)
   ✅ 기존 파일들이 있는 상태에서 실행
✅ UNIFIED_CITY_INFO 확인: 116개 도시

🚀 폴더 구조 생성 실행...
🛡️ 안전한 계층적 폴더 구조 생성 시작...
📋 원칙: 기존 파일들은 절대 건드리지 않음

📁 Step 1: 기본 구조 생성...
  ✅ config/ 생성 완료
  ✅ data/ 생성 완료
  ✅ data/history/ 생성 완료
  ✅ temp/ 생성 완료
  ✅ temp/cookies/ 생성 완료

🌍 Step 2: 대륙/국가 계층 구조 생성...
📊 총 116개 도시에서 대륙/국가 정보 추출...
🗺️ 발견된 대륙/국가 조합: 45개
  🌏 북미/ 생성 완료
    🏳️ 북미/멕시코/ 생성 완료
    🏳️ 북미/미국/ 생성 완료
    🏳️ 북미/캐나다/ 생성 완료
  🌏 아시아/ 생성 완료
    🏳️ 아시아/네팔/ 생성 완료
    🏳️ 아시아/대만/ 생성 완료
    🏳️ 아시아/대한민국/ 생성 완료
    🏳️ 아시아/라오스/ 생성 완료
    🏳️ 아시아/마카오/ 생성 완료
    🏳️ 아시아/말레이시아/ 생성 완료
    🏳️ 아시아/베트남/ 생성 완료
    🏳️ 아시아/싱가포르/ 생성 완료
    🏳️ 아시아/아랍에미리트/ 생성 완료
    🏳️ 아시아/인도/ 생성 완료
    🏳️ 아시아/인도네시아/ 생성 완료
    🏳️ 아시아/일본/ 생성 완료
    🏳️ 아시아/중국/ 생성 완료
    🏳️ 아시아/캄보디아/ 생성 완료
    🏳️ 아시아/태국/ 생성 완료
    🏳️ 아시아/필리핀/ 생성 완료
    🏳️ 아시아/홍콩/ 생성 완료
  

In [38]:
#1-2 테스트중
# =============================================================================
# 🧹 기존 파일들을 새 구조로 깔끔하게 통합
# 테스트 단계니까 과감하게 체계적으로 정리!
# =============================================================================

import os
import shutil
import json
import glob
from datetime import datetime

def integrate_existing_files():
    """
    기존 테스트 파일들을 새로운 계층적 구조로 이동
    - CSV 파일들 → data/대륙/국가/ 이동
    - 메타데이터 → config/ 이동  
    - 쿠키 → temp/ 이동
    - 백업 생성 후 안전하게 이동
    """
    print("🧹 기존 파일들을 새 구조로 깔끔하게 통합 시작!")
    print("📋 원칙: 백업 생성 → 안전한 이동 → 검증")
    
    moved_files = []
    errors = []
    
    try:
        # 1. 백업 폴더 생성
        print("\n💾 Step 1: 안전을 위한 백업 생성...")
        backup_folder = f"backup_before_integration_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
        os.makedirs(backup_folder, exist_ok=True)
        print(f"   ✅ 백업 폴더 생성: {backup_folder}")
        
        # 2. CSV 파일들 이동
        print("\n📄 Step 2: CSV 파일들을 계층 구조로 이동...")
        csv_files = glob.glob("myrealtrip_*.csv")
        
        for csv_file in csv_files:
            try:
                # 백업 생성
                shutil.copy2(csv_file, backup_folder)
                print(f"   💾 백업: {csv_file}")
                
                # 파일명에서 도시 추출
                city_name = extract_city_from_filename(csv_file)
                if city_name and city_name in UNIFIED_CITY_INFO:
                    continent = UNIFIED_CITY_INFO[city_name]["대륙"]
                    country = UNIFIED_CITY_INFO[city_name]["국가"]
                    
                    # 새로운 파일명 생성
                    new_filename = f"myrealtrip_{city_name}_complete.csv"
                    new_path = f"data/{continent}/{country}/{new_filename}"
                    
                    # 폴더 생성 확인
                    os.makedirs(os.path.dirname(new_path), exist_ok=True)
                    
                    # 파일 이동
                    shutil.move(csv_file, new_path)
                    moved_files.append(f"{csv_file} → {new_path}")
                    print(f"   ✅ 이동: {csv_file} → {new_path}")
                else:
                    print(f"   ⚠️ 도시 인식 실패: {csv_file} (수동 처리 필요)")
                    
            except Exception as e:
                error_msg = f"CSV 이동 실패 {csv_file}: {e}"
                errors.append(error_msg)
                print(f"   ❌ {error_msg}")
        
        # 3. 메타데이터 파일 이동
        print("\n📊 Step 3: 메타데이터 파일 이동...")
        metadata_files = ["data_metadata.json"]
        
        for metadata_file in metadata_files:
            if os.path.exists(metadata_file):
                try:
                    # 백업
                    shutil.copy2(metadata_file, backup_folder)
                    
                    # config 폴더로 이동
                    new_path = f"config/{metadata_file}"
                    shutil.move(metadata_file, new_path)
                    moved_files.append(f"{metadata_file} → {new_path}")
                    print(f"   ✅ 이동: {metadata_file} → {new_path}")
                    
                except Exception as e:
                    error_msg = f"메타데이터 이동 실패 {metadata_file}: {e}"
                    errors.append(error_msg)
                    print(f"   ❌ {error_msg}")
        
        # 4. 쿠키 폴더 이동
        print("\n🍪 Step 4: 쿠키 폴더 이동...")
        if os.path.exists("cookies"):
            try:
                # 백업 (폴더 전체)
                shutil.copytree("cookies", f"{backup_folder}/cookies")
                
                # temp 폴더로 이동
                if os.path.exists("temp/cookies"):
                    shutil.rmtree("temp/cookies")  # 기존 것 삭제
                
                shutil.move("cookies", "temp/cookies")
                moved_files.append("cookies/ → temp/cookies/")
                print(f"   ✅ 이동: cookies/ → temp/cookies/")
                
            except Exception as e:
                error_msg = f"쿠키 폴더 이동 실패: {e}"
                errors.append(error_msg)
                print(f"   ❌ {error_msg}")
        
        # 5. 이동 결과 검증
        print("\n🔍 Step 5: 이동 결과 검증...")
        verify_integration()
        
        # 6. 결과 요약
        print(f"\n📊 통합 결과:")
        print(f"   ✅ 이동된 파일: {len(moved_files)}개")
        print(f"   ❌ 오류: {len(errors)}개")
        print(f"   💾 백업 위치: {backup_folder}")
        
        if len(errors) == 0:
            print(f"\n🎉 깔끔한 통합 완료!")
            print(f"📁 모든 파일이 새로운 계층 구조로 이동됨")
        else:
            print(f"\n⚠️ 일부 오류 발생:")
            for error in errors:
                print(f"   - {error}")
        
        return {
            "success": len(errors) == 0,
            "moved_files": moved_files,
            "errors": errors,
            "backup_folder": backup_folder
        }
        
    except Exception as e:
        error_msg = f"통합 프로세스 전체 실패: {e}"
        print(f"\n❌ {error_msg}")
        return {
            "success": False,
            "moved_files": moved_files,
            "errors": [error_msg],
            "backup_folder": backup_folder
        }

def extract_city_from_filename(filename):
    """파일명에서 도시명 추출"""
    try:
        # myrealtrip_푸켓_products_2개_20250716_134318.csv
        # → 푸켓 추출
        parts = filename.replace(".csv", "").split("_")
        if len(parts) >= 2:
            potential_city = parts[1]
            # UNIFIED_CITY_INFO에서 확인
            if potential_city in UNIFIED_CITY_INFO:
                return potential_city
    except:
        pass
    return None

def verify_integration():
    """통합 결과 검증"""
    print("   🔍 새 구조 파일 존재 확인...")
    
    # data 폴더 내 CSV 파일 확인
    csv_found = 0
    for continent in ["아시아", "유럽", "북미", "오세아니아"]:
        continent_path = f"data/{continent}"
        if os.path.exists(continent_path):
            for country in os.listdir(continent_path):
                country_path = f"{continent_path}/{country}"
                if os.path.isdir(country_path):
                    csv_files = glob.glob(f"{country_path}/*.csv")
                    csv_found += len(csv_files)
    
    print(f"   📄 새 구조 CSV 파일: {csv_found}개 발견")
    
    # config 폴더 확인
    config_files = 0
    if os.path.exists("config"):
        config_files = len([f for f in os.listdir("config") if f.endswith('.json')])
    
    print(f"   📊 config 폴더 JSON 파일: {config_files}개 발견")
    
    # temp/cookies 확인
    cookies_exists = os.path.exists("temp/cookies")
    print(f"   🍪 temp/cookies 폴더: {'✅ 존재' if cookies_exists else '❌ 없음'}")

def show_new_structure():
    """새로운 구조 시각화"""
    print("\n🗂️ 통합 후 새로운 구조:")
    print("=" * 60)
    
    try:
        print("📁 mikael_project/ (깔끔해진 루트)")
        print("├── 📄 test_4.ipynb")
        print("├── 📁 config/")
        
        # config 폴더 내용
        if os.path.exists("config"):
            config_files = [f for f in os.listdir("config") if f.endswith('.json')]
            for i, file in enumerate(config_files):
                prefix = "│   └──" if i == len(config_files)-1 else "│   ├──"
                print(f"{prefix} 📄 {file}")
        
        print("├── 📁 data/ (계층적 데이터)")
        
        # data 폴더 구조 (일부만 표시)
        if os.path.exists("data"):
            continents = [f for f in os.listdir("data") 
                         if os.path.isdir(f"data/{f}") and f != "history"][:2]  # 처음 2개만
            
            for i, continent in enumerate(continents):
                is_last = i == len(continents) - 1
                prefix = "│   └──" if is_last else "│   ├──"
                print(f"{prefix} 📁 {continent}/")
                
                # 국가 폴더 (첫 번째만)
                continent_path = f"data/{continent}"
                if os.path.exists(continent_path):
                    countries = [f for f in os.listdir(continent_path) 
                               if os.path.isdir(f"{continent_path}/{f}")][:1]
                    
                    for country in countries:
                        if is_last:
                            print(f"       └── 📁 {country}/")
                        else:
                            print(f"│      └── 📁 {country}/")
                        
                        # CSV 파일 확인
                        country_path = f"{continent_path}/{country}"
                        csv_files = glob.glob(f"{country_path}/*.csv")
                        if csv_files:
                            csv_name = os.path.basename(csv_files[0])
                            if is_last:
                                print(f"           └── 📄 {csv_name}")
                            else:
                                print(f"│          └── 📄 {csv_name}")
        
        print("├── 📁 temp/")
        if os.path.exists("temp/cookies"):
            print("│   └── 📁 cookies/")
        
        print("└── 📁 myrealtripthumb_img/ (그대로 유지)")
        
        print("=" * 60)
        print("🎯 결과: 모든 파일이 체계적으로 정리됨!")
        
    except Exception as e:
        print(f"❌ 구조 표시 실패: {e}")

def test_clean_integration():
    """깔끔한 통합 테스트"""
    print("🧪 기존 파일들 깔끔한 통합 테스트")
    print("=" * 60)
    
    # 1. 현재 루트 파일 확인
    print("1️⃣ 통합 전 루트 폴더 상태...")
    root_files = [f for f in os.listdir(".") if os.path.isfile(f)]
    csv_files = [f for f in root_files if f.startswith("myrealtrip_") and f.endswith(".csv")]
    json_files = [f for f in root_files if f.endswith(".json")]
    
    print(f"   📄 CSV 파일: {len(csv_files)}개")
    print(f"   📊 JSON 파일: {len(json_files)}개")
    print(f"   🍪 cookies 폴더: {os.path.exists('cookies')}")
    
    # 2. 통합 실행
    print("\n2️⃣ 파일 통합 실행...")
    result = integrate_existing_files()
    
    # 3. 통합 후 상태 확인
    print("\n3️⃣ 통합 후 상태 확인...")
    new_root_files = [f for f in os.listdir(".") if os.path.isfile(f)]
    remaining_csv = [f for f in new_root_files if f.startswith("myrealtrip_") and f.endswith(".csv")]
    
    print(f"   📄 루트 남은 CSV: {len(remaining_csv)}개")
    print(f"   🗂️ 새 구조로 이동 완료: {len(result['moved_files'])}개")
    
    # 4. 결과 출력
    if result["success"]:
        print("\n🎉 깔끔한 통합 성공!")
        show_new_structure()
        print(f"\n🎯 다음 단계: Step 3 - 통합된 크롤링 시스템 구축")
    else:
        print(f"\n⚠️ 통합 중 일부 오류 발생")
        print(f"💾 백업 위치: {result['backup_folder']}")
    
    return result

# =============================================================================
# 실행 지침 및 자동 실행
# =============================================================================

print("=" * 70)
print("🧹 기존 파일들 깔끔한 통합 시스템")
print("=" * 70)
print("📋 통합 내용:")
print("   📄 CSV 파일들 → data/대륙/국가/ 이동")
print("   📊 메타데이터 → config/ 이동")
print("   🍪 cookies → temp/ 이동")
print("   💾 모든 파일 백업 생성")
print("")
print("🎯 결과:")
print("   ✅ 체계적인 계층 구조")
print("   ✅ 깔끔한 루트 폴더")
print("   ✅ 안전한 백업 보존")
print("=" * 70)

print("\n🚀 자동 실행 시작!\n")

# 자동 통합 실행
result = test_clean_integration()

if result["success"]:
    print("\n🎉 Step 2.5 완료! 기존 파일들 깔끔하게 통합!")
    print("🗂️ 이제 완벽한 계층적 구조로 정리됨!")
else:
    print("\n⚠️ 일부 오류 발생. 백업에서 복구 가능.")

print("\n🎯 다음: 작전 일지 시스템 테스트 후 Step 3 진행")

🧹 기존 파일들 깔끔한 통합 시스템
📋 통합 내용:
   📄 CSV 파일들 → data/대륙/국가/ 이동
   📊 메타데이터 → config/ 이동
   🍪 cookies → temp/ 이동
   💾 모든 파일 백업 생성

🎯 결과:
   ✅ 체계적인 계층 구조
   ✅ 깔끔한 루트 폴더
   ✅ 안전한 백업 보존

🚀 자동 실행 시작!

🧪 기존 파일들 깔끔한 통합 테스트
1️⃣ 통합 전 루트 폴더 상태...
   📄 CSV 파일: 0개
   📊 JSON 파일: 0개
   🍪 cookies 폴더: True

2️⃣ 파일 통합 실행...
🧹 기존 파일들을 새 구조로 깔끔하게 통합 시작!
📋 원칙: 백업 생성 → 안전한 이동 → 검증

💾 Step 1: 안전을 위한 백업 생성...
   ✅ 백업 폴더 생성: backup_before_integration_20250718_002446

📄 Step 2: CSV 파일들을 계층 구조로 이동...

📊 Step 3: 메타데이터 파일 이동...

🍪 Step 4: 쿠키 폴더 이동...
   ❌ 쿠키 폴더 이동 실패: [('cookies\\85\\Default\\Cache\\Cache_Data\\data_0', 'backup_before_integration_20250718_002446/cookies\\85\\Default\\Cache\\Cache_Data\\data_0', "[Errno 13] Permission denied: 'cookies\\\\85\\\\Default\\\\Cache\\\\Cache_Data\\\\data_0'"), ('cookies\\85\\Default\\Cache\\Cache_Data\\data_1', 'backup_before_integration_20250718_002446/cookies\\85\\Default\\Cache\\Cache_Data\\data_1', "[Errno 13] Permission denied: 'cookies\\\\85\\\\Default\\\\Cache\\\\Cach

In [39]:
#1-3 테스트중
# =============================================================================
# 🎯 Step 3: 작전 일지(crawling_state.json) 시스템
# "절대 멈추지 않는 지능형 크롤러"의 핵심 - 메모리 시스템
# =============================================================================

import json
import os
import glob
from datetime import datetime

class CrawlingStateManager:
    """
    크롤링 상태 관리 시스템
    - 모든 진행상황을 실시간으로 기록
    - 중단 시 마지막 지점부터 재시작 가능
    - "블랙박스" 역할 수행
    """
    
    def __init__(self, config_path="config/crawling_state.json"):
        self.config_path = config_path
        self.state = {}
        print(f"🎯 작전 일지 시스템 초기화: {config_path}")
        self.load_state()
    
    def load_state(self):
        """작전 일지 로드 또는 신규 생성"""
        try:
            if os.path.exists(self.config_path):
                with open(self.config_path, 'r', encoding='utf-8') as f:
                    self.state = json.load(f)
                print(f"📖 기존 작전 일지 발견! 마지막 임무 인계받음")
                print(f"   📊 총 수집: {self.state.get('progress', {}).get('total_collected_count', 0)}개")
                print(f"   📄 마지막 페이지: {self.state.get('progress', {}).get('last_worked_page', 1)}")
                print(f"   🎯 진행 중인 도시: {self.state.get('progress', {}).get('current_city', '없음')}")
                print(f"   ✅ 완료 URL: {len(self.state.get('progress', {}).get('completed_urls', []))}개")
            else:
                print(f"📝 새로운 작전 일지 생성!")
                self.create_new_state()
                
        except Exception as e:
            print(f"⚠️ 작전 일지 로드 실패, 새로 생성: {e}")
            self.create_new_state()
    
    def create_new_state(self):
        """새로운 작전 일지 생성"""
        self.state = {
            "mission_info": {
                "mission_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "version": "1.0",
                "status": "initialized",
                "description": "절대 멈추지 않는 지능형 크롤러"
            },
            "progress": {
                "total_collected_count": 0,
                "last_worked_page": 1,
                "current_city": None,
                "completed_urls": [],
                "failed_urls": [],
                "current_session_count": 0
            },
            "cities": {},
            "statistics": {
                "total_sessions": 0,
                "total_errors": 0,
                "total_pages_completed": 0,
                "average_per_page": 0,
                "success_rate": 1.0,
                "last_restart_count": 0
            },
            "current_session": {
                "session_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "session_id": f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
                "products_this_session": 0,
                "pages_this_session": 0
            }
        }
        self.save_state()
        print(f"✅ 새 작전 일지 생성 완료: {self.config_path}")
    
    def save_state(self):
        """✅ 작전 일지 저장 - UTF-8 강화 + 안정성 개선"""
        try:
            # 🔥 핵심 수정 1: 폴더 강제 생성
            config_dir = os.path.dirname(self.config_path)
            if config_dir:  # 경로에 폴더가 있는 경우만
                os.makedirs(config_dir, exist_ok=True)
            
            # 🔥 핵심 수정 2: 마지막 업데이트 시간 갱신
            self.state["mission_info"]["last_update"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # 🔥 핵심 수정 3: 저장 전 데이터 검증
            if not isinstance(self.state, dict):
                print("⚠️ state 데이터가 올바르지 않음")
                return False
                
            # 🔥 핵심 수정 4: UTF-8 강제 저장 (BOM 없음, JSON용)
            with open(self.config_path, 'w', encoding='utf-8') as f:
                json.dump(self.state, f, ensure_ascii=False, indent=2)
            
            # 🔥 핵심 수정 5: 저장 성공 확인
            if os.path.exists(self.config_path):
                file_size = os.path.getsize(self.config_path)
                if file_size > 10:  # 최소 10바이트 이상이어야 정상
                    return True
                else:
                    print(f"⚠️ 저장된 파일이 너무 작음: {file_size} bytes")
                    
        except Exception as e:
            print(f"❌ 작전 일지 저장 실패: {e}")
            
            # 🔥 핵심 수정 6: 긴급 백업 시스템
            try:
                backup_timestamp = datetime.now().strftime('%H%M%S')
                backup_path = f"backup_crawling_state_{backup_timestamp}.json"
                
                with open(backup_path, 'w', encoding='utf-8') as f:
                    json.dump(self.state, f, ensure_ascii=False, indent=2)
                
                print(f"🚨 작전 일지 긴급 백업 성공: {backup_path}")
                print(f"💡 나중에 {backup_path}를 {self.config_path}로 이동하세요!")
                return True  # 백업이라도 성공했으므로 True
                
            except Exception as backup_error:
                print(f"🚨 긴급 백업도 실패: {backup_error}")
                return False
        
        return False

    
    def start_city_mission(self, city_name):
        """새로운 도시 임무 시작"""
        print(f"🎯 새 임무 시작: {city_name}")
        
        # 현재 진행상황 업데이트
        self.state["progress"]["current_city"] = city_name
        self.state["progress"]["last_worked_page"] = 1
        self.state["progress"]["current_session_count"] = 0
        
        # 새 세션 시작
        self.state["current_session"] = {
            "session_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "session_id": f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
            "products_this_session": 0,
            "pages_this_session": 0,
            "target_city": city_name
        }
        
        # 도시별 기록 초기화 (기존 기록이 있으면 이어서)
        if city_name not in self.state["cities"]:
            self.state["cities"][city_name] = {
                "total_products": 0,
                "first_started": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "last_session": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "completed_urls": [],
                "failed_urls": [],
                "pages_completed": [],
                "status": "in_progress",
                "sessions": []
            }
        else:
            # 기존 도시 재시작
            self.state["cities"][city_name]["status"] = "resumed"
            self.state["cities"][city_name]["last_session"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        self.state["mission_info"]["status"] = "active"
        self.save_state()
        print(f"✅ {city_name} 임무 등록 완료")
    
    def record_product_success(self, city_name, product_url, product_data):
        """상품 수집 성공 기록 (실시간) - 핵심 기능"""
        # 전체 카운터 증가
        self.state["progress"]["total_collected_count"] += 1
        self.state["progress"]["current_session_count"] += 1
        
        # 현재 세션 카운터 증가
        self.state["current_session"]["products_this_session"] += 1
        
        # URL 완료 목록에 추가 (중복 방지)
        if product_url not in self.state["progress"]["completed_urls"]:
            self.state["progress"]["completed_urls"].append(product_url)
        
        # 도시별 기록 업데이트
        if city_name in self.state["cities"]:
            self.state["cities"][city_name]["total_products"] += 1
            self.state["cities"][city_name]["last_session"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            if product_url not in self.state["cities"][city_name]["completed_urls"]:
                self.state["cities"][city_name]["completed_urls"].append(product_url)
        
        # 상태 업데이트
        self.state["mission_info"]["status"] = "active"
        
        # 즉시 저장 (가장 중요!)
        self.save_state()
        
        total = self.state['progress']['total_collected_count']
        city_total = self.state['cities'][city_name]['total_products']
        print(f"📊 진행: 전체 {total}개 | {city_name}: {city_total}개 | 세션: {self.state['current_session']['products_this_session']}개")
    
    def record_product_failure(self, city_name, product_url, error_msg):
        """상품 수집 실패 기록"""
        # 실패 기록 생성
        failure_record = {
            "url": product_url,
            "error": str(error_msg),
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "city": city_name
        }
        
        # 전체 실패 목록에 추가
        self.state["progress"]["failed_urls"].append(failure_record)
        self.state["statistics"]["total_errors"] += 1
        
        # 도시별 실패 기록
        if city_name in self.state["cities"]:
            self.state["cities"][city_name]["failed_urls"].append(failure_record)
        
        self.save_state()
        print(f"⚠️ 수집 실패 기록: {product_url[:50]}... | 오류: {error_msg}")
    
    def complete_page(self, city_name, page_number, page_product_count):
        """페이지 완료 기록"""
        self.state["progress"]["last_worked_page"] = page_number
        self.state["current_session"]["pages_this_session"] += 1
        self.state["statistics"]["total_pages_completed"] += 1
        
        # 도시별 페이지 완료 기록
        if city_name in self.state["cities"]:
            page_record = {
                "page": page_number,
                "products": page_product_count,
                "completed_at": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "session_id": self.state["current_session"]["session_id"]
            }
            self.state["cities"][city_name]["pages_completed"].append(page_record)
        
        # 평균 계산 업데이트
        if self.state["statistics"]["total_pages_completed"] > 0:
            self.state["statistics"]["average_per_page"] = round(
                self.state["progress"]["total_collected_count"] / self.state["statistics"]["total_pages_completed"], 2
            )
        
        self.save_state()
        print(f"📄 페이지 {page_number} 완료: {page_product_count}개 수집 | 평균: {self.state['statistics']['average_per_page']}개/페이지")
    
    def record_browser_restart(self, city_name, restart_reason="scheduled"):
        """브라우저 재시작 기록"""
        restart_record = {
            "timestamp": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "reason": restart_reason,
            "products_before_restart": self.state["current_session"]["products_this_session"],
            "city": city_name
        }
        
        # 재시작 기록 추가
        if "browser_restarts" not in self.state:
            self.state["browser_restarts"] = []
        self.state["browser_restarts"].append(restart_record)
        
        self.state["statistics"]["last_restart_count"] = self.state["current_session"]["products_this_session"]
        
        self.save_state()
        print(f"🔄 브라우저 재시작 기록: {restart_reason} | 재시작 전 수집: {restart_record['products_before_restart']}개")
    
    def complete_city_mission(self, city_name):
        """도시 임무 완료"""
        if city_name in self.state["cities"]:
            self.state["cities"][city_name]["status"] = "completed"
            self.state["cities"][city_name]["completed_at"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # 세션 기록 저장
            session_summary = {
                "session_id": self.state["current_session"]["session_id"],
                "products": self.state["current_session"]["products_this_session"],
                "pages": self.state["current_session"]["pages_this_session"],
                "duration": self.state["current_session"]["session_start"]
            }
            self.state["cities"][city_name]["sessions"].append(session_summary)
        
        self.state["progress"]["current_city"] = None
        self.state["statistics"]["total_sessions"] += 1
        
        # 성공률 계산
        total_attempts = len(self.state["progress"]["completed_urls"]) + len(self.state["progress"]["failed_urls"])
        if total_attempts > 0:
            self.state["statistics"]["success_rate"] = round(
                len(self.state["progress"]["completed_urls"]) / total_attempts, 3
            )
        
        self.save_state()
        print(f"🎉 {city_name} 임무 완료! 성공률: {self.state['statistics']['success_rate']*100}%")
    
    def is_url_completed(self, url):
        """✅ URL 중복 확인 - 정규화 강화 버전"""
        try:
            # 🔥 핵심 수정 1: URL 정규화 (쿼리 파라미터 제거)
            import urllib.parse
            parsed_url = urllib.parse.urlparse(url)
            clean_url = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
            
            # 🔥 핵심 수정 2: 정규화된 URL들과 비교
            completed_urls = self.state.get("progress", {}).get("completed_urls", [])
            
            for completed_url in completed_urls:
                try:
                    parsed_completed = urllib.parse.urlparse(completed_url)
                    clean_completed = f"{parsed_completed.scheme}://{parsed_completed.netloc}{parsed_completed.path}"
                    
                    if clean_url == clean_completed:
                        return True
                except:
                    # 정규화 실패 시 기존 방식으로 폴백
                    if url == completed_url:
                        return True
            
            return False
            
        except Exception as e:
            print(f"⚠️ URL 중복 체크 실패: {e}")
            # 🔥 핵심 수정 3: 에러 시 안전한 폴백
            completed_urls = self.state.get("progress", {}).get("completed_urls", [])
            return url in completed_urls
    
    def get_resume_info(self):
        """재시작 정보 반환"""
        return {
            "last_city": self.state["progress"]["current_city"],
            "last_page": self.state["progress"]["last_worked_page"],
            "total_collected": self.state["progress"]["total_collected_count"],
            "completed_urls": self.state["progress"]["completed_urls"],
            "session_count": self.state["progress"]["current_session_count"]
        }
    
    def get_city_progress(self, city_name):
        """특정 도시의 진행상황 반환"""
        if city_name in self.state["cities"]:
            return self.state["cities"][city_name]
        return None
    
    def show_status(self):
        """현재 상태 출력"""
        print("\n📊 현재 작전 상황:")
        print("=" * 60)
        
        # 기본 정보
        print(f"🎯 진행 중인 도시: {self.state['progress']['current_city']}")
        print(f"📄 마지막 페이지: {self.state['progress']['last_worked_page']}")
        print(f"📊 총 수집 개수: {self.state['progress']['total_collected_count']}개")
        print(f"🔄 현재 세션: {self.state['progress']['current_session_count']}개")
        print(f"✅ 완료 URL 수: {len(self.state['progress']['completed_urls'])}개")
        print(f"❌ 실패 URL 수: {len(self.state['progress']['failed_urls'])}개")
        
        # 통계 정보
        stats = self.state["statistics"]
        print(f"\n📈 통계 정보:")
        print(f"   📊 총 세션: {stats['total_sessions']}회")
        print(f"   📄 완료 페이지: {stats['total_pages_completed']}페이지")
        print(f"   📊 페이지당 평균: {stats['average_per_page']}개")
        print(f"   ✅ 성공률: {stats['success_rate']*100:.1f}%")
        
        # 도시별 현황
        if self.state["cities"]:
            print(f"\n🏙️ 도시별 현황:")
            for city, info in self.state["cities"].items():
                status_emoji = "🎯" if info["status"] == "in_progress" else "🔄" if info["status"] == "resumed" else "✅"
                print(f"   {status_emoji} {city}: {info['total_products']}개 | {info['status']}")
        
        # 현재 세션 정보
        session = self.state["current_session"]
        print(f"\n🔄 현재 세션:")
        print(f"   🆔 세션 ID: {session['session_id']}")
        print(f"   📊 세션 수집: {session['products_this_session']}개")
        print(f"   📄 세션 페이지: {session['pages_this_session']}페이지")
        
        print("=" * 60)

def test_crawling_state_system():
    """작전 일지 시스템 전체 테스트"""
    print("🧪 작전 일지 시스템 종합 테스트")
    print("=" * 70)
    
    try:
        # 1. 상태 관리자 초기화
        print("1️⃣ 상태 관리자 초기화...")
        state_manager = CrawlingStateManager()
        
        # 2. 테스트 시나리오: 푸켓 임무
        print("\n2️⃣ 테스트 시나리오: 푸켓 임무 시작...")
        state_manager.start_city_mission("푸켓")
        
        # 3. 가상의 상품 수집 시뮬레이션
        print("\n3️⃣ 가상 상품 수집 시뮬레이션...")
        test_urls = [
            "https://experiences.myrealtrip.com/products/test1",
            "https://experiences.myrealtrip.com/products/test2", 
            "https://experiences.myrealtrip.com/products/test3"
        ]
        
        for i, url in enumerate(test_urls, 1):
            product_data = {
                "name": f"푸켓 테스트 상품 {i}", 
                "price": f"{i*15000}원",
                "rating": f"{4.0 + i*0.2}"
            }
            state_manager.record_product_success("푸켓", url, product_data)
        
        # 4. 페이지 완료 및 브라우저 재시작 시뮬레이션
        print("\n4️⃣ 페이지 완료 및 재시작 시뮬레이션...")
        state_manager.complete_page("푸켓", 1, 3)
        state_manager.record_browser_restart("푸켓", "테스트 재시작")
        
        # 5. 실패 케이스 시뮬레이션
        print("\n5️⃣ 실패 케이스 시뮬레이션...")
        state_manager.record_product_failure("푸켓", "https://test-fail.com", "타임아웃 오류")
        
        # 6. 상태 확인
        print("\n6️⃣ 현재 작전 상황 확인...")
        state_manager.show_status()
        
        # 7. 재시작 정보 확인
        print("\n7️⃣ 재시작 정보 확인...")
        resume_info = state_manager.get_resume_info()
        print(f"   🔄 재시작 시 도시: {resume_info['last_city']}")
        print(f"   📄 재시작 시 페이지: {resume_info['last_page']}")
        print(f"   📊 누적 수집: {resume_info['total_collected']}개")
        print(f"   ✅ 완료 URL: {len(resume_info['completed_urls'])}개")
        
        # 8. URL 중복 체크 테스트
        print("\n8️⃣ URL 중복 체크 테스트...")
        test_url = test_urls[0]
        is_completed = state_manager.is_url_completed(test_url)
        print(f"   🔍 {test_url[:50]}...")
        print(f"   ✅ 이미 완료됨: {is_completed}")
        
        # 9. 파일 생성 확인
        print("\n9️⃣ 작전 일지 파일 확인...")
        if os.path.exists("config/crawling_state.json"):
            print("   ✅ config/crawling_state.json 생성 확인")
            
            # 파일 크기 확인
            file_size = os.path.getsize("config/crawling_state.json")
            print(f"   📊 파일 크기: {file_size:,} bytes")
            
            # 내용 미리보기
            with open("config/crawling_state.json", 'r', encoding='utf-8') as f:
                state_content = json.load(f)
                print(f"   📊 파일 내 총 수집: {state_content['progress']['total_collected_count']}개")
                print(f"   🎯 파일 내 현재 도시: {state_content['progress']['current_city']}")
        else:
            print("   ❌ 작전 일지 파일 생성 실패")
            return False
        
        # 10. 기존 CSV 파일과 연동 확인
        print("\n🔟 기존 데이터와 연동성 확인...")
        csv_files = glob.glob("data/아시아/태국/*.csv")
        if csv_files:
            print(f"   ✅ 기존 CSV 파일: {len(csv_files)}개 발견")
            for csv_file in csv_files:
                print(f"      📄 {os.path.basename(csv_file)}")
        else:
            print("   ℹ️ 기존 CSV 파일 없음 (정상 - 새로운 크롤링 시 생성됨)")
        
        print(f"\n🎉 작전 일지 시스템 종합 테스트 완료!")
        print(f"🎯 결과: 절대 멈추지 않는 크롤러의 핵심 시스템 구축 성공!")
        print(f"🔄 이제 크롤링이 어디서 중단되어도 정확히 이어받기 가능!")
        return True
        
    except Exception as e:
        print(f"\n❌ 테스트 실패: {type(e).__name__}: {e}")
        import traceback
        traceback.print_exc()
        return False

# =============================================================================
# 실행 지침 및 자동 실행
# =============================================================================

print("=" * 80)
print("🎯 작전 일지(crawling_state.json) 시스템")
print("=" * 80)
print("📋 핵심 기능:")
print("   🔄 실시간 크롤링 진행상황 기록")
print("   🎯 중단 시 마지막 지점부터 재시작")
print("   ✅ URL 중복 처리 방지")
print("   📊 도시별 상세 통계 및 성과 추적")
print("   🔄 브라우저 재시작 기록 및 관리")
print("   📈 세션별 수집 현황 모니터링")
print("")
print("🗂️ 생성되는 파일:")
print("   📄 config/crawling_state.json - 모든 상태 정보")
print("")
print("🎯 핵심 원리:")
print("   - 상품 하나 수집할 때마다 즉시 기록")
print("   - 브라우저 종료되어도 상태 완벽 보존")
print("   - 재시작 시 completed_urls로 중복 방지")
print("   - 페이지/도시/전체 단위 진행상황 추적")
print("=" * 80)

print("\n🚀 자동 실행 시작!\n")

# 자동 테스트 실행
success = test_crawling_state_system()

if success:
    print("\n🎉 Step 3 완료! 작전 일지 시스템 구축 성공!")
    print("📋 주요 성과:")
    print("   ✅ 실시간 상태 추적 시스템 구축")
    print("   ✅ 중단/재시작 연속성 보장")
    print("   ✅ URL 중복 처리 방지 시스템")
    print("   ✅ 다층 백업 및 통계 시스템")
    print("")
    print("🔄 이제 크롤링이 중단되어도:")
    print("   📊 마지막 수집 상품까지 모든 기록 보존")
    print("   🎯 정확한 지점부터 재시작")
    print("   ✅ 중복 수집 자동 방지")
    print("   📈 실시간 진행상황 모니터링")
else:
    print("\n⚠️ Step 3 부분 실패. 문제 확인 필요.")

print("\n🎯 다음: Step 4 - 기존 크롤링 시스템과 통합")
print("💡 이제 '절대 멈추지 않는 지능형 크롤러'의 핵심이 완성되었습니다!")

🎯 작전 일지(crawling_state.json) 시스템
📋 핵심 기능:
   🔄 실시간 크롤링 진행상황 기록
   🎯 중단 시 마지막 지점부터 재시작
   ✅ URL 중복 처리 방지
   📊 도시별 상세 통계 및 성과 추적
   🔄 브라우저 재시작 기록 및 관리
   📈 세션별 수집 현황 모니터링

🗂️ 생성되는 파일:
   📄 config/crawling_state.json - 모든 상태 정보

🎯 핵심 원리:
   - 상품 하나 수집할 때마다 즉시 기록
   - 브라우저 종료되어도 상태 완벽 보존
   - 재시작 시 completed_urls로 중복 방지
   - 페이지/도시/전체 단위 진행상황 추적

🚀 자동 실행 시작!

🧪 작전 일지 시스템 종합 테스트
1️⃣ 상태 관리자 초기화...
🎯 작전 일지 시스템 초기화: config/crawling_state.json
📖 기존 작전 일지 발견! 마지막 임무 인계받음
   📊 총 수집: 6개
   📄 마지막 페이지: 1
   🎯 진행 중인 도시: 푸켓
   ✅ 완료 URL: 3개

2️⃣ 테스트 시나리오: 푸켓 임무 시작...
🎯 새 임무 시작: 푸켓
✅ 푸켓 임무 등록 완료

3️⃣ 가상 상품 수집 시뮬레이션...
📊 진행: 전체 7개 | 푸켓: 7개 | 세션: 1개
📊 진행: 전체 8개 | 푸켓: 8개 | 세션: 2개
📊 진행: 전체 9개 | 푸켓: 9개 | 세션: 3개

4️⃣ 페이지 완료 및 재시작 시뮬레이션...
📄 페이지 1 완료: 3개 수집 | 평균: 3.0개/페이지
🔄 브라우저 재시작 기록: 테스트 재시작 | 재시작 전 수집: 3개

5️⃣ 실패 케이스 시뮬레이션...
⚠️ 수집 실패 기록: https://test-fail.com... | 오류: 타임아웃 오류

6️⃣ 현재 작전 상황 확인...

📊 현재 작전 상황:
🎯 진행 중인 도시: 푸켓
📄 마지막 페이지: 1
📊 총 수집 개수: 9개
🔄 현재 세션: 3개
✅ 완료 URL 수: 3개
❌ 실패 URL 수: 4개

📈 통계 정보:
   📊

In [34]:
#1-4
# =============================================================================
# 🚀 Step 4: 절대 멈추지 않는 지능형 크롤러 (최종 통합)
# 기존 크롤링 시스템 + 작전 일지 시스템 + 계층적 저장 구조
# =============================================================================

import random
import time
import pandas as pd
from datetime import datetime
import os

class IntelligentCrawler:
    """
    절대 멈추지 않는 지능형 크롤러
    - 작전 일지 시스템과 완전 통합
    - 페이지 단위 순회 + 랜덤 재시작
    - 계층적 데이터 저장
    """
    
    def __init__(self, city_name):
        self.city_name = city_name
        self.continent, self.country = get_city_info(city_name)
        
        # 작전 일지 시스템 초기화
        self.state_manager = CrawlingStateManager()
        
        # 랜덤 재시작 설정
        self.restart_after = random.randint(7, 20)
        self.current_count = 0
        
        print(f"🎯 지능형 크롤러 초기화: {city_name}")
        print(f"🌍 위치: {self.continent} > {self.country}")
        print(f"🔄 재시작 예정: {self.restart_after}개 후")
    
    def start_mission(self):
        """임무 시작"""
        print(f"\n🚀 {self.city_name} 지능형 크롤링 임무 시작!")
        print("=" * 60)
        
        # 작전 일지에 임무 등록
        self.state_manager.start_city_mission(self.city_name)
        
        # 재시작 정보 확인
        resume_info = self.state_manager.get_resume_info()
        if resume_info['last_city'] == self.city_name and resume_info['total_collected'] > 0:
            print(f"🔄 임무 재개: 이전 수집 {resume_info['total_collected']}개")
            print(f"📄 마지막 페이지: {resume_info['last_page']}")
        else:
            print(f"🆕 새로운 임무 시작")
        
        return True
    
    def collect_products_from_urls(self, urls_to_crawl, driver):
        """URL 목록에서 상품 정보 수집 (작전 일지 연동)"""
        print(f"\n📊 수집 시작: {len(urls_to_crawl)}개 URL")
        
        all_results = []
        
        for product_index, product_url in enumerate(urls_to_crawl, 1):
            self.current_count += 1
            
            # 중복 URL 체크 (작전 일지)
            if self.state_manager.is_url_completed(product_url):
                print(f"  ⏭️ 상품 {product_index}: 이미 완료된 URL 스킵")
                continue
            
            # 🔧 수정: 브라우저 재시작 체크 및 driver 업데이트
            if self.current_count >= self.restart_after:
                print(f"\n🔄 {self.current_count}개 수집 완료! 브라우저 재시작...")
                driver = self.perform_browser_restart(driver)  # driver 업데이트
                self.current_count = 0
                self.restart_after = random.randint(7, 20)
                print(f"🎯 다음 재시작: {self.restart_after}개 후")
            
            print(f"\n📦 상품 {product_index}/{len(urls_to_crawl)} 처리 중...")
            print(f"🔗 URL: {product_url[:60]}...")
            
            # 기본값 설정
            product_name = "정보 없음"
            price_raw = "정보 없음" 
            price_clean = "정보 없음"
            rating_raw = "정보 없음"
            rating_clean = "정보 없음"
            review_count = ""
            language = ""
            img_info = {'status': '처리 안됨', 'filename': '', 'path': '', 'relative_path': '', 'size': 0}
            url_type = "Product" if "/products/" in product_url else "Offer"
            
            try:
                # 🔧 원본 파일의 단순한 URL 이동 로직 (검증됨)
                print(f"  🔗 상품 {product_index} URL로 이동 중: {product_url}")
                driver.get(product_url)
                time.sleep(random.uniform(CONFIG["MIN_DELAY"], CONFIG["MAX_DELAY"]))
                
                # 기존 그룹 1 함수들을 사용하여 정보 수집
                product_name = get_product_name(driver, url_type)
                price_raw = get_price(driver)
                price_clean = clean_price(price_raw)
                rating_raw = get_rating(driver)
                rating_clean = clean_rating(rating_raw)
                review_count = get_review_count(driver)
                language = get_language(driver)
                
                # 이미지 다운로드 (계층적 구조)
                img_info = download_image(driver, product_name, self.city_name, 
                                        self.state_manager.state["progress"]["total_collected_count"] + 1)
                
                # 결과 데이터 생성
                result = {
                    '번호': len(all_results) + 1,
                    '대륙': self.continent,
                    '국가': self.country, 
                    '도시': self.city_name,
                    '공항코드': get_city_code(self.city_name),
                    '상품번호': product_index,
                    '상품타입': url_type,
                    '상품명': product_name,
                    '가격_원본': price_raw,
                    '가격_정제': price_clean,
                    '평점_원본': rating_raw,
                    '평점_정제': rating_clean,
                    '리뷰수': review_count,
                    '언어': language,
                    '이미지_파일명': img_info.get('filename', ''),
                    '이미지_상대경로': img_info.get('relative_path', ''),
                    '이미지_전체경로': img_info.get('path', ''),
                    '이미지_상태': img_info.get('status', ''),
                    '이미지_크기': img_info.get('size', 0),
                    'URL': product_url,
                    '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
                    '상태': '완전수집'
                }
                
                all_results.append(result)
                
                # 🎯 핵심: 작전 일지에 성공 기록 (즉시!)
                self.state_manager.record_product_success(self.city_name, product_url, result)
                
                # 계층적 구조로 즉시 저장 (append)
                self.save_to_hierarchical_structure(result)
                
                print(f"  ✅ 수집 완료: {product_name[:30]}...")
                
            except Exception as e:
                print(f"  ❌ 수집 실패: {type(e).__name__}: {e}")
                
                # 실패도 작전 일지에 기록
                self.state_manager.record_product_failure(self.city_name, product_url, str(e))
                
                # 실패한 상품도 기록 (디버깅용)
                result = {
                    '번호': len(all_results) + 1,
                    '대륙': self.continent,
                    '국가': self.country,
                    '도시': self.city_name,
                    '공항코드': get_city_code(self.city_name),
                    '상품번호': product_index,
                    '상품타입': url_type,
                    '상품명': f"수집실패_{product_index}",
                    '가격_원본': "수집실패",
                    '가격_정제': "수집실패",
                    '평점_원본': "수집실패",
                    '평점_정제': "수집실패",
                    '리뷰수': "",
                    '언어': "",
                    '이미지_파일명': "",
                    '이미지_상대경로': "",
                    '이미지_전체경로': "",
                    '이미지_상태': "처리실패",
                    '이미지_크기': 0,
                    'URL': product_url,
                    '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
                    '상태': f'수집실패({type(e).__name__})'
                }
                all_results.append(result)
                continue
        
        return all_results
    
    def save_to_hierarchical_structure(self, result):
        """계층적 구조로 데이터 저장 (append 방식)"""
        try:
            # 계층적 경로 생성
            city_csv_path = f"data/{self.continent}/{self.country}/myrealtrip_{self.city_name}_complete.csv"
            
            # 폴더 생성 확인
            os.makedirs(os.path.dirname(city_csv_path), exist_ok=True)
            
            # DataFrame 생성
            df = pd.DataFrame([result])
            
            # 파일이 있으면 append, 없으면 새로 생성
            if os.path.exists(city_csv_path):
                df.to_csv(city_csv_path, mode='a', header=False, index=False, encoding='utf-8-sig')
            else:
                df.to_csv(city_csv_path, index=False, encoding='utf-8-sig')
            
            # history 폴더에도 세션별 백업 (선택적)
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            session_id = self.state_manager.state["current_session"]["session_id"]
            history_path = f"data/history/{timestamp}_{self.city_name}_{session_id}_incremental.csv"
            
            # history 폴더 생성 및 저장
            os.makedirs("data/history", exist_ok=True)
            df.to_csv(history_path, mode='a', header=not os.path.exists(history_path), 
                     index=False, encoding='utf-8-sig')
            
        except Exception as e:
            print(f"⚠️ 계층적 저장 실패: {e}")
    
    def perform_browser_restart(self, driver):
        """🔧 검색 상태 복원이 포함된 안전한 브라우저 재시작"""
        try:
            print("  🔄 브라우저 재시작 시작...")
            
            # 작전 일지에 재시작 기록
            self.state_manager.record_browser_restart(self.city_name, "scheduled_restart")
            
            # 안전한 종료
            driver.quit()
            
            # 대기
            restart_delay = random.uniform(3, 8)
            print(f"  ⏰ {restart_delay:.1f}초 대기 중...")
            time.sleep(restart_delay)
            
            # 새 브라우저 시작
            print("  🚀 새 브라우저 시작...")
            new_driver = setup_driver()
            
            # 🔧 핵심 수정: 검색 상태 복원
            print(f"  🔍 {self.city_name} 검색 상태 복원 중...")
            try:
                # 메인 페이지 이동
                go_to_main_page(new_driver)
                
                # 검색 실행
                find_and_fill_search(new_driver, self.city_name)
                click_search_button(new_driver)
                
                # 팝업 처리
                try:
                    handle_popup(new_driver)
                except:
                    pass
                
                # 전체 보기
                try:
                    click_view_all(new_driver)
                except:
                    pass
                
                print("  ✅ 검색 상태 복원 완료!")
                
            except Exception as search_error:
                print(f"  ⚠️ 검색 상태 복원 실패: {search_error}")
                print("  💡 기본 상태로 계속 진행...")
            
            print("  ✅ 브라우저 재시작 완료!")
            return new_driver
            
        except Exception as e:
            print(f"  ❌ 브라우저 재시작 실패: {e}")
            # 실패 시 기존 드라이버 유지
            return driver
    
    def complete_mission(self):
        """임무 완료 처리"""
        print(f"\n🎉 {self.city_name} 임무 완료!")
        
        # 작전 일지에 완료 기록
        self.state_manager.complete_city_mission(self.city_name)
        
        # 최종 상태 출력
        self.state_manager.show_status()
        
        # 최종 파일 정보
        city_csv_path = f"data/{self.continent}/{self.country}/myrealtrip_{self.city_name}_complete.csv"
        if os.path.exists(city_csv_path):
            file_size = os.path.getsize(city_csv_path)
            print(f"\n📊 최종 결과:")
            print(f"   📄 CSV 파일: {city_csv_path}")
            print(f"   📊 파일 크기: {file_size:,} bytes")
        
        return True

def run_intelligent_crawler(city_name):
    """지능형 크롤러 실행 (통합 버전)"""
    print("🚀 절대 멈추지 않는 지능형 크롤러 시작!")
    print("=" * 80)
    
    try:
        # 1. 지능형 크롤러 초기화
        crawler = IntelligentCrawler(city_name)
        
        # 2. 임무 시작
        crawler.start_mission()
        
        # 3. 브라우저 설정 및 시작
        print(f"\n🌐 브라우저 설정 중...")
        driver = setup_driver()
        
        # 4. 기존 그룹 2 로직: 검색 및 페이지 이동
        print(f"\n🔍 {city_name} 상품 검색 중...")
        
        # 메인 페이지 이동
        go_to_main_page(driver)
        
        # 검색 실행
        find_and_fill_search(driver, city_name)
        click_search_button(driver)
        
        # 팝업 처리
        try:
            handle_popup(driver)
        except:
            pass
        
        # 전체 보기
        try:
            click_view_all(driver)
        except:
            pass
        
        # 5. 기존 그룹 3 로직: URL 수집
        print(f"\n📊 상품 URL 수집 중...")
        collected_urls = collect_all_24_urls(driver)
        
        if not collected_urls:
            print("❌ 수집할 URL이 없습니다")
            return False
        
        # 설정된 개수만큼만 크롤링
        max_products = CONFIG['MAX_PRODUCTS_PER_CITY']
        urls_to_crawl = collected_urls[:max_products]
        
        print(f"🎯 크롤링 대상: {len(urls_to_crawl)}개 (전체 {len(collected_urls)}개 중)")
        
        # 6. 지능형 크롤링 실행
        results = crawler.collect_products_from_urls(urls_to_crawl, driver)
        
        # 7. 임무 완료
        crawler.complete_mission()
        
        # 8. 브라우저 정리
        driver.quit()
        
        print(f"\n🎉 지능형 크롤러 완료!")
        print(f"📊 총 수집: {len([r for r in results if r['상태'] == '완전수집'])}개")
        print(f"💾 저장 위치: data/{crawler.continent}/{crawler.country}/")
        
        return True
        
    except Exception as e:
        print(f"\n❌ 지능형 크롤러 실패: {type(e).__name__}: {e}")
        
        # 오류 시에도 브라우저 정리
        try:
            driver.quit()
        except:
            pass
            
        return False

def test_intelligent_crawler():
    """지능형 크롤러 테스트"""
    print("🧪 지능형 크롤러 테스트 실행")
    print("=" * 80)
    
    # 테스트할 도시 (CITIES_TO_SEARCH에서 가져오기)
    if not CITIES_TO_SEARCH:
        print("❌ CITIES_TO_SEARCH가 비어있습니다!")
        print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
        return False
    
    test_city = CITIES_TO_SEARCH[0]
    print(f"🎯 테스트 도시: {test_city}")
    
    # 지능형 크롤러 실행
    success = run_intelligent_crawler(test_city)
    
    if success:
        print(f"\n🎉 지능형 크롤러 테스트 완료!")
        print(f"🎯 결과: '절대 멈추지 않는 지능형 크롤러' 완성!")
        print(f"\n✅ 달성된 기능:")
        print(f"   🔄 실시간 상태 추적 및 저장")
        print(f"   🎯 중단 시 정확한 지점부터 재시작")
        print(f"   📊 URL 중복 수집 방지")
        print(f"   🗂️ 계층적 구조로 체계적 저장") 
        print(f"   🔄 랜덤 브라우저 재시작 (7~20개)")
        print(f"   📈 다층 통계 및 모니터링")
    else:
        print(f"\n⚠️ 테스트 실패. 설정 확인 필요.")
    
    return success

# =============================================================================
# 실행 지침 및 자동 실행
# =============================================================================

print("=" * 90)
print("🚀 절대 멈추지 않는 지능형 크롤러 (최종 완성판)")
print("=" * 90)
print("🎯 통합된 기능:")
print("   ✅ 기존 그룹 1-4 모든 크롤링 기능")
print("   ✅ 작전 일지 시스템 (실시간 상태 추적)")
print("   ✅ 계층적 데이터 저장 구조")
print("   ✅ 중단/재시작 연속성 보장")
print("   ✅ URL 중복 수집 방지")
print("   ✅ 랜덤 브라우저 재시작 (7~20개)")
print("   ✅ 다층 백업 및 통계 시스템")
print("   ✅ 브라우저 재시작 시 검색 상태 복원")
print("")
print("🗂️ 저장 구조:")
print("   📄 data/대륙/국가/myrealtrip_도시_complete.csv (누적)")
print("   📄 data/history/세션별_백업.csv (세션별)")
print("   📄 config/crawling_state.json (상태 추적)")
print("")
print("🎯 핵심 원리:")
print("   - 상품 하나 수집할 때마다 즉시 3곳에 저장")
print("   - 브라우저 종료되어도 상태 완벽 보존")
print("   - 재시작 시 completed_urls로 중복 완전 방지")
print("   - 7~20개마다 랜덤 재시작으로 탐지 회피")
print("   - 재시작 시 검색 상태 자동 복원")
print("=" * 90)

print("\n🚀 자동 실행 시작!\n")

# 최종 테스트 실행
success = test_intelligent_crawler()

if success:
    print("\n" + "🎉" * 20)
    print("🏆 1단계: 기능 완성 달성!")
    print("🎯 '절대 멈추지 않는 지능형 크롤러' 완전 구축!")
    print("🔄 이제 어떤 상황에서도 중단 없이 크롤링 가능!")
    print("📊 실시간 모니터링과 완벽한 복구 시스템 완성!")
    print("🎉" * 20)
else:
    print("\n⚠️ 최종 테스트 실패. 이전 단계들을 확인해주세요.")

print("\n🎯 다음 단계: 2단계 리팩토링 (선택사항)")
print("💡 현재 상태로도 완전한 대용량 크롤링 시스템입니다!")

🚀 절대 멈추지 않는 지능형 크롤러 (최종 완성판)
🎯 통합된 기능:
   ✅ 기존 그룹 1-4 모든 크롤링 기능
   ✅ 작전 일지 시스템 (실시간 상태 추적)
   ✅ 계층적 데이터 저장 구조
   ✅ 중단/재시작 연속성 보장
   ✅ URL 중복 수집 방지
   ✅ 랜덤 브라우저 재시작 (7~20개)
   ✅ 다층 백업 및 통계 시스템
   ✅ 브라우저 재시작 시 검색 상태 복원

🗂️ 저장 구조:
   📄 data/대륙/국가/myrealtrip_도시_complete.csv (누적)
   📄 data/history/세션별_백업.csv (세션별)
   📄 config/crawling_state.json (상태 추적)

🎯 핵심 원리:
   - 상품 하나 수집할 때마다 즉시 3곳에 저장
   - 브라우저 종료되어도 상태 완벽 보존
   - 재시작 시 completed_urls로 중복 완전 방지
   - 7~20개마다 랜덤 재시작으로 탐지 회피
   - 재시작 시 검색 상태 자동 복원

🚀 자동 실행 시작!

🧪 지능형 크롤러 테스트 실행
🎯 테스트 도시: 푸켓
🚀 절대 멈추지 않는 지능형 크롤러 시작!
🎯 작전 일지 시스템 초기화: config/crawling_state.json
📖 기존 작전 일지 발견! 마지막 임무 인계받음
   📊 총 수집: 6개
   📄 마지막 페이지: 1
   🎯 진행 중인 도시: 푸켓
   ✅ 완료 URL: 3개
🎯 지능형 크롤러 초기화: 푸켓
🌍 위치: 아시아 > 태국
🔄 재시작 예정: 20개 후

🚀 푸켓 지능형 크롤링 임무 시작!
🎯 새 임무 시작: 푸켓
✅ 푸켓 임무 등록 완료
🔄 임무 재개: 이전 수집 6개
📄 마지막 페이지: 1

🌐 브라우저 설정 중...
✅ 크롬 드라이버 실행 성공!
Windows

🔍 푸켓 상품 검색 중...
  🔍 '푸켓' 검색창 찾는 중...
  ✅ 검색창을 찾았습니다!
  📝 '푸켓' 키워드 입력 완료
  🔎 검색 버튼 찾는 중...
  ✅ 검색 버튼 클릭 성공!
  ✅ 팝업창을 닫았습니다.
 

KeyboardInterrupt: 

In [1]:
## url테스트1
## 🚀 그룹 2: 브라우저 시작 + 푸켓 검색
print("🔄 그룹 2: 브라우저 시작 + 검색 복원")
print("=" * 50)

# 브라우저 시작
driver = setup_driver()

# 푸켓 검색 과정
city_name = CITIES_TO_SEARCH[0]
print(f"🔍 {city_name} 검색 시작...")

# 메인 페이지 이동
go_to_main_page(driver)

# 검색 실행
find_and_fill_search(driver, city_name)
click_search_button(driver)

# 팝업 처리
try:
    handle_popup(driver)
except:
    pass

# 전체 보기
try:
    click_view_all(driver)
except:
    pass

print(f"✅ {city_name} 검색 완료!")



# 🔧 OFFER 상품 포함 전체 URL 수집
print("🔧 OFFER 상품 포함 전체 URL 수집")
print("=" * 50)

# 현재 페이지 확인
current_url = driver.current_url
print(f"현재 페이지: {current_url}")

# 모든 링크 요소 찾기
all_links = driver.find_elements(By.TAG_NAME, "a")
print(f"전체 링크 요소: {len(all_links)}개")

# 🎯 수정된 로직: Products + Offers 모두 수집
all_product_urls = []

for link in all_links:
    try:
        href = link.get_attribute('href')
        if href and (
            'experiences.myrealtrip.com/products/' in href or  # 기존 Products
            'myrealtrip.com/offers/' in href                   # 새로 추가: Offers
        ):
            # 중복 제거
            if href not in all_product_urls:
                all_product_urls.append(href)
    except:
        continue

print(f"\n📊 수집 결과:")
print(f"   ✅ 전체 상품 URL: {len(all_product_urls)}개")

# Products와 Offers 분리 집계
products_count = len([url for url in all_product_urls if 'products/' in url])
offers_count = len([url for url in all_product_urls if 'offers/' in url])

print(f"   📦 Products: {products_count}개")
print(f"   🎯 Offers: {offers_count}개")
print(f"   🎉 총합: {products_count + offers_count}개")

# 수집된 URL 샘플 출력
if all_product_urls:
    print(f"\n📋 수집된 URL 샘플 (처음 10개):")
    for i, url in enumerate(all_product_urls[:10], 1):
        url_type = "Product" if 'products/' in url else "Offer"
        print(f"{i}. [{url_type}] {url}")
    
    if len(all_product_urls) > 10:
        print(f"   ... (총 {len(all_product_urls)}개)")
    
    # collected_urls 업데이트
    collected_urls = all_product_urls
    print(f"\n🔄 collected_urls 업데이트: {len(collected_urls)}개")
    
    # 24개 목표 달성 여부 확인
    if len(collected_urls) >= 24:
        print("🎉 24개 목표 달성!")
    elif len(collected_urls) >= 20:
        print("✅ 20개 이상 수집 성공!")
    else:
        print(f"⚠️ 목표 24개보다 적음 ({len(collected_urls)}개)")
        
else:
    print("❌ 상품 URL을 찾을 수 없습니다")

print(f"\n🎯 다음 단계: {len(all_product_urls)}개 URL로 상품 정보 수집")



# 🧪 URL 진단 테스트
print("🧪 URL 진단 테스트 시작")
print("=" * 50)

if 'collected_urls' in globals() and collected_urls:
    # 현재 브라우저 상태 확인
    current_url = driver.current_url
    print(f"현재 브라우저 위치: {current_url}")
    
    # 첫 번째 URL 테스트
    test_url = collected_urls[0]
    print(f"\n🧪 첫 번째 URL 테스트:")
    print(f"대상 URL: {test_url}")
    
    # 직접 이동
    driver.get(test_url)
    time.sleep(5)
    
    # 결과 확인
    final_url = driver.current_url
    print(f"도착 URL: {final_url}")
    
    if test_url in final_url:
        print("✅ 정상 이동됨")
    else:
        print("❌ 리다이렉트 발생!")
        print(f"예상: {test_url}")
        print(f"실제: {final_url}")
else:
    print("❌ collected_urls가 없습니다. 2단계를 먼저 실행하세요.")


# 🚀 최종 크롤링 테스트
print("🚀 최종 크롤링 테스트")
print("=" * 50)

# 테스트용으로 처음 3개 URL만 크롤링 (시간 절약)
test_urls = collected_urls[:3]
print(f"🎯 테스트 대상: {len(test_urls)}개 URL")

# 결과 저장용 리스트
test_results = []

# 각 URL에서 상품 정보 수집
for i, url in enumerate(test_urls, 1):
    print(f"\n📦 상품 {i}/{len(test_urls)} 처리 중...")
    
    # URL 타입 확인
    url_type = "Product" if 'products/' in url else "Offer"
    print(f"🔗 [{url_type}] {url[:60]}...")
    
    try:
        # URL로 이동
        driver.get(url)
        time.sleep(random.uniform(3, 6))
        
        # 기본 정보 수집 (URL 타입에 따라 다른 함수 호출)
        product_name = get_product_name(driver, url_type)
        price_raw = get_price(driver)
        price_clean = clean_price(price_raw)
        
        # 추가 정보 수집 (선택적)
        try:
            rating_raw = get_rating(driver)
            rating_clean = clean_rating(rating_raw)
            review_count = get_review_count(driver)
        except:
            rating_raw = "수집실패"
            rating_clean = "수집실패"
            review_count = ""
        
        # 결과 저장
        result = {
            '번호': i,
            '상품타입': url_type,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            'URL': url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '성공'
        }
        test_results.append(result)
        
        print(f"  ✅ 수집 완료:")
        print(f"     상품명: {product_name[:40]}...")
        print(f"     가격: {price_clean}")
        print(f"     평점: {rating_clean}")
        
    except Exception as e:
        print(f"  ❌ 수집 실패: {type(e).__name__}: {e}")
        
        # 실패한 경우도 기록
        result = {
            '번호': i,
            '상품타입': url_type,
            '상품명': f"수집실패_{i}",
            '가격_원본': "수집실패",
            '가격_정제': "수집실패",
            '평점_원본': "수집실패",
            '평점_정제': "수집실패",
            '리뷰수': "",
            'URL': url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': f'실패({type(e).__name__})'
        }
        test_results.append(result)

# 결과 출력
print(f"\n📊 테스트 결과:")
print(f"   총 처리: {len(test_results)}개")
success_count = len([r for r in test_results if r['상태'] == '성공'])
print(f"   성공: {success_count}개")
print(f"   실패: {len(test_results) - success_count}개")

# 결과 테이블 출력
if test_results:
    print(f"\n📋 수집 결과 요약:")
    print("-" * 80)
    for result in test_results:
        print(f"{result['번호']}. [{result['상품타입']}] {result['상품명'][:30]}... | {result['가격_정제']} | {result['상태']}")
    print("-" * 80)

# 성공 시 다음 단계 안내
if success_count > 0:
    print(f"\n🎉 크롤링 테스트 성공!")
    print(f"💡 이제 전체 24개 URL로 본격적인 크롤링을 진행할 수 있습니다!")
    print(f"🎯 다음 단계: 전체 크롤링 시스템 실행")
else:
    print(f"\n⚠️ 크롤링 테스트 실패. 문제 분석 필요.")

print(f"\n🔄 collected_urls 상태: {len(collected_urls)}개 준비 완료")    

🔄 그룹 2: 브라우저 시작 + 검색 복원


NameError: name 'setup_driver' is not defined

In [ ]:

# 🔧 OFFER 상품 포함 전체 URL 수집
print("🔧 OFFER 상품 포함 전체 URL 수집")
print("=" * 50)

# 현재 페이지 확인
current_url = driver.current_url
print(f"현재 페이지: {current_url}")

# 모든 링크 요소 찾기
all_links = driver.find_elements(By.TAG_NAME, "a")
print(f"전체 링크 요소: {len(all_links)}개")

# 🎯 수정된 로직: Products + Offers 모두 수집
all_product_urls = []

for link in all_links:
    try:
        href = link.get_attribute('href')
        if href and (
            'experiences.myrealtrip.com/products/' in href or  # 기존 Products
            'myrealtrip.com/offers/' in href                   # 새로 추가: Offers
        ):
            # 중복 제거
            if href not in all_product_urls:
                all_product_urls.append(href)
    except:
        continue

print(f"\n📊 수집 결과:")
print(f"   ✅ 전체 상품 URL: {len(all_product_urls)}개")

# Products와 Offers 분리 집계
products_count = len([url for url in all_product_urls if 'products/' in url])
offers_count = len([url for url in all_product_urls if 'offers/' in url])

print(f"   📦 Products: {products_count}개")
print(f"   🎯 Offers: {offers_count}개")
print(f"   🎉 총합: {products_count + offers_count}개")

# 수집된 URL 샘플 출력
if all_product_urls:
    print(f"\n📋 수집된 URL 샘플 (처음 10개):")
    for i, url in enumerate(all_product_urls[:10], 1):
        url_type = "Product" if 'products/' in url else "Offer"
        print(f"{i}. [{url_type}] {url}")
    
    if len(all_product_urls) > 10:
        print(f"   ... (총 {len(all_product_urls)}개)")
    
    # collected_urls 업데이트
    collected_urls = all_product_urls
    print(f"\n🔄 collected_urls 업데이트: {len(collected_urls)}개")
    
    # 24개 목표 달성 여부 확인
    if len(collected_urls) >= 24:
        print("🎉 24개 목표 달성!")
    elif len(collected_urls) >= 20:
        print("✅ 20개 이상 수집 성공!")
    else:
        print(f"⚠️ 목표 24개보다 적음 ({len(collected_urls)}개)")
        
else:
    print("❌ 상품 URL을 찾을 수 없습니다")

print(f"\n🎯 다음 단계: {len(all_product_urls)}개 URL로 상품 정보 수집")

🔧 OFFER 상품 포함 전체 URL 수집
현재 페이지: https://www.myrealtrip.com/offers?t=llp&qct=Phuket&qcr=Thailand
전체 링크 요소: 52개

📊 수집 결과:
   ✅ 전체 상품 URL: 24개
   📦 Products: 18개
   🎯 Offers: 6개
   🎉 총합: 24개

📋 수집된 URL 샘플 (처음 10개):
1. [Product] https://experiences.myrealtrip.com/products/3446452
2. [Product] https://experiences.myrealtrip.com/products/3446456
3. [Product] https://experiences.myrealtrip.com/products/3147846
4. [Product] https://experiences.myrealtrip.com/products/3446495
5. [Product] https://experiences.myrealtrip.com/products/3442583
6. [Offer] https://www.myrealtrip.com/offers/120576
7. [Product] https://experiences.myrealtrip.com/products/3887414
8. [Offer] https://www.myrealtrip.com/offers/148629
9. [Product] https://experiences.myrealtrip.com/products/3887426
10. [Product] https://experiences.myrealtrip.com/products/3446446
   ... (총 24개)

🔄 collected_urls 업데이트: 24개
🎉 24개 목표 달성!

🎯 다음 단계: 24개 URL로 상품 정보 수집


In [ ]:
# 🧪 URL 진단 테스트
print("🧪 URL 진단 테스트 시작")
print("=" * 50)

if 'collected_urls' in globals() and collected_urls:
    # 현재 브라우저 상태 확인
    current_url = driver.current_url
    print(f"현재 브라우저 위치: {current_url}")
    
    # 첫 번째 URL 테스트
    test_url = collected_urls[0]
    print(f"\n🧪 첫 번째 URL 테스트:")
    print(f"대상 URL: {test_url}")
    
    # 직접 이동
    driver.get(test_url)
    time.sleep(5)
    
    # 결과 확인
    final_url = driver.current_url
    print(f"도착 URL: {final_url}")
    
    if test_url in final_url:
        print("✅ 정상 이동됨")
    else:
        print("❌ 리다이렉트 발생!")
        print(f"예상: {test_url}")
        print(f"실제: {final_url}")
else:
    print("❌ collected_urls가 없습니다. 2단계를 먼저 실행하세요.")

🧪 URL 진단 테스트 시작
현재 브라우저 위치: https://www.myrealtrip.com/offers?t=llp&qct=Phuket&qcr=Thailand

🧪 첫 번째 URL 테스트:
대상 URL: https://experiences.myrealtrip.com/products/3446452
도착 URL: https://experiences.myrealtrip.com/products/3446452
✅ 정상 이동됨


In [52]:
# 🚀 최종 크롤링 테스트
print("🚀 최종 크롤링 테스트")
print("=" * 50)

# 테스트용으로 처음 3개 URL만 크롤링 (시간 절약)
test_urls = collected_urls[:3]
print(f"🎯 테스트 대상: {len(test_urls)}개 URL")

# 결과 저장용 리스트
test_results = []

# 각 URL에서 상품 정보 수집
for i, url in enumerate(test_urls, 1):
    print(f"\n📦 상품 {i}/{len(test_urls)} 처리 중...")
    
    # URL 타입 확인
    url_type = "Product" if 'products/' in url else "Offer"
    print(f"🔗 [{url_type}] {url[:60]}...")
    
    try:
        # URL로 이동
        driver.get(url)
        time.sleep(random.uniform(3, 6))
        
        # 기본 정보 수집 (URL 타입에 따라 다른 함수 호출)
        product_name = get_product_name(driver, url_type)
        price_raw = get_price(driver)
        price_clean = clean_price(price_raw)
        
        # 추가 정보 수집 (선택적)
        try:
            rating_raw = get_rating(driver)
            rating_clean = clean_rating(rating_raw)
            review_count = get_review_count(driver)
        except:
            rating_raw = "수집실패"
            rating_clean = "수집실패"
            review_count = ""
        
        # 결과 저장
        result = {
            '번호': i,
            '상품타입': url_type,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            'URL': url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '성공'
        }
        test_results.append(result)
        
        print(f"  ✅ 수집 완료:")
        print(f"     상품명: {product_name[:40]}...")
        print(f"     가격: {price_clean}")
        print(f"     평점: {rating_clean}")
        
    except Exception as e:
        print(f"  ❌ 수집 실패: {type(e).__name__}: {e}")
        
        # 실패한 경우도 기록
        result = {
            '번호': i,
            '상품타입': url_type,
            '상품명': f"수집실패_{i}",
            '가격_원본': "수집실패",
            '가격_정제': "수집실패",
            '평점_원본': "수집실패",
            '평점_정제': "수집실패",
            '리뷰수': "",
            'URL': url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': f'실패({type(e).__name__})'
        }
        test_results.append(result)

# 결과 출력
print(f"\n📊 테스트 결과:")
print(f"   총 처리: {len(test_results)}개")
success_count = len([r for r in test_results if r['상태'] == '성공'])
print(f"   성공: {success_count}개")
print(f"   실패: {len(test_results) - success_count}개")

# 결과 테이블 출력
if test_results:
    print(f"\n📋 수집 결과 요약:")
    print("-" * 80)
    for result in test_results:
        print(f"{result['번호']}. [{result['상품타입']}] {result['상품명'][:30]}... | {result['가격_정제']} | {result['상태']}")
    print("-" * 80)

# 성공 시 다음 단계 안내
if success_count > 0:
    print(f"\n🎉 크롤링 테스트 성공!")
    print(f"💡 이제 전체 24개 URL로 본격적인 크롤링을 진행할 수 있습니다!")
    print(f"🎯 다음 단계: 전체 크롤링 시스템 실행")
else:
    print(f"\n⚠️ 크롤링 테스트 실패. 문제 분석 필요.")

print(f"\n🔄 collected_urls 상태: {len(collected_urls)}개 준비 완료")

🚀 최종 크롤링 테스트
🎯 테스트 대상: 3개 URL

📦 상품 1/3 처리 중...
🔗 [Product] https://experiences.myrealtrip.com/products/3446452...
  📊 Product 상품명 수집 중...
  💰 가격 정보 수집 중...
    ✅ 유효한 가격 발견: '52,000원~'
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 196개
  ✅ 수집 완료:
     상품명: [푸켓/일일투어] 피피섬+마야베이+필레라군+ 몽키비치+카이섬 스피드 보트...
     가격: 52,000원
     평점: 4.3

📦 상품 2/3 처리 중...
🔗 [Product] https://experiences.myrealtrip.com/products/3446456...
  📊 Product 상품명 수집 중...
  💰 가격 정보 수집 중...
    ✅ 유효한 가격 발견: '22,000원~'
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 97개
  ✅ 수집 완료:
     상품명: 푸켓 시티투어 10시간+공항 샌딩+차량 기사 포함(단독차량)...
     가격: 22,000원
     평점: 4.7

📦 상품 3/3 처리 중...
🔗 [Product] https://experiences.myrealtrip.com/products/3147846...
  📊 Product 상품명 수집 중...
  💰 가격 정보 수집 중...
    ✅ 유효한 가격 발견: '89,000원~'
  📝 리뷰 수 정보 찾는 중...
  ✅ 리뷰 수 정보 발견: 후기 106개
  ✅ 수집 완료:
     상품명: 🤿[드론포함/푸켓의 몰디브!] 라차섬+바나나비치 스피드보트투어(체험다이빙...
     가격: 89,000원
     평점: 4.8

📊 테스트 결과:
   총 처리: 3개
   성공: 3개
   실패: 0개

📋 수집 결과 요약:
----------------------------------------